# Observation and Next step

Based on the baseline analysis, it was observed that meta-llama/Meta-Llama-3.1-8B-Instruct perfromance was better than other models in the list for both task. So, keeping this model fixed, we tried with multistage detection. The reason for selecting multistage was that the labels were long tailed. Some labels had high frequency, some had mid range frequency and some had very low frequency. Using these frequencies, different labels were introduced at different stages.

# TSV

### Single stage

In [1]:
import sys
import json
import pandas as pd
from collections import defaultdict, Counter
from typing import List, Dict, Set, Tuple
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

class OneStageClassifier:
    
    STAGE_1 = [
        "Appeal-to-emotion", "Loaded-language", "Vagueness", 
        "Name-calling-or-labelling", "Doubt", "Hasty-generalization", 
        "Evading-the-burden-of-proof", "Flag-waving", "Ad-hominem", "False-analogy", 
        "Slogan", "Thought-terminating-cliches", "Red-herring", 
        "Appeal-to-authority", "Slippery-slope", "Causal-oversimplification", "Strawman", 
        "Cherry-picking", "False-dilemma", "Circular-reasoning"
    ]
    
    ALL_LABELS = STAGE_1 

    def __init__(self, model_name="meta-llama/Meta-Llama-3.1-8B-Instruct", use_cpu=False, use_4bit=False):
        self.device = "cuda" if torch.cuda.is_available() and not use_cpu else "cpu"
        print(f"Loading model on device: {self.device}")
    
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
    
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def _format_for_model(self, prompt: str) -> str:
        messages = [{"role": "user", "content": prompt}]
        if hasattr(self.tokenizer, "apply_chat_template"):
            try:
                return self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            except Exception:
                return prompt
        return prompt

    def _generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.0) -> str:
        text = self._format_for_model(prompt)
        inputs = self.tokenizer(text, return_tensors="pt", padding=True).to(self.device)
        
        with torch.inference_mode():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=0.95,
                do_sample=True if temperature > 0 else False,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id or self.tokenizer.eos_token_id,
            )
        
        generated = outputs[0][inputs["input_ids"].shape[1]:]
        response = self.tokenizer.decode(generated, skip_special_tokens=True)
        
        del inputs, outputs, generated
        torch.cuda.empty_cache()
        
        return response

    def _extract_labels_from_response(self, response: str, valid_labels: List[str]) -> List[str]:
        labels = []
        
        # Strategy 1: Try JSON format
        try:
            start = response.find("{")
            end = response.rfind("}") + 1
            if start != -1 and end > start:
                json_str = response[start:end]
                data = json.loads(json_str)
                if "labels" in data:
                    labels = data["labels"]
                elif "fallacies" in data:
                    labels = data["fallacies"]
        except:
            pass
        
        # Strategy 2: Try JSON array
        if not labels:
            try:
                start = response.find("[")
                end = response.rfind("]") + 1
                if start != -1 and end > start:
                    json_str = response[start:end]
                    labels = json.loads(json_str)
            except:
                pass
        
        # Strategy 3: Extract labels mentioned in text
        if not labels:
            for label in valid_labels:
                if label in response:
                    labels.append(label)
        
        # Clean and validate labels
        valid_output = []
        for label in labels:
            label = label.strip()
            if label in valid_labels:
                valid_output.append(label)
            else:
                normalized = label.replace(" ", "-")
                if normalized in valid_labels:
                    valid_output.append(normalized)
        
        return list(set(valid_output))

    def build_few_shot_examples(self, train_df: pd.DataFrame, 
                                 labels_subset: List[str],
                                 n_examples_per_label: int = 3,
                                 labels_col: str = "labels_a1",
                                 max_text_length: int = 300) -> Dict[str, List[str]]:
        examples = defaultdict(list)
        
        train_df_shuffled = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        for idx, row in train_df_shuffled.iterrows():
            labels = str(row[labels_col]).split('|')
            labels = [l.strip() for l in labels if l.strip() and l.strip() in labels_subset]
            
            if not labels:
                continue
            
            text = row['post_text']
            if len(text) > max_text_length:
                text = text[:max_text_length] + "..."
            
            for label in labels:
                if label in labels_subset and len(examples[label]) < n_examples_per_label:
                    examples[label].append(text)
            
            if all(len(examples[label]) >= n_examples_per_label for label in labels_subset):
                break
        
        print(f"\n  Built examples for {len(examples)} labels in this stage:")
        for label in labels_subset:
            count = len(examples.get(label, []))
            print(f"    {label}: {count} examples")
        
        return dict(examples)

    def create_stage_prompt(self, text: str, stage_labels: List[str], 
                           examples: Dict[str, List[str]], 
                           stage_name: str,
                           max_examples: int = 3) -> str:
        
        label_descriptions = {
            "Ad-hominem": "Attacking the person instead of the argument",
            "Appeal-to-authority": "Using authority as evidence",
            "Appeal-to-emotion": "Manipulating emotions instead of using logic",
            "Causal-oversimplification": "Oversimplifying cause-effect relationships",
            "Cherry-picking": "Selecting only favorable evidence",
            "Circular-reasoning": "Conclusion is assumed in the premise",
            "Doubt": "Questioning credibility without evidence",
            "Evading-the-burden-of-proof": "Avoiding providing evidence",
            "False-analogy": "Comparing incomparable things",
            "False-dilemma": "Presenting only two options when more exist",
            "Flag-waving": "Appealing to nationalism/patriotism",
            "Hasty-generalization": "Drawing conclusions from insufficient evidence",
            "Loaded-language": "Using emotionally charged words",
            "Name-calling-or-labelling": "Using derogatory labels",
            "Red-herring": "Introducing irrelevant information",
            "Slippery-slope": "Claiming one thing will lead to extreme consequences",
            "Slogan": "Using catchy phrases instead of arguments",
            "Strawman": "Misrepresenting opponent's argument",
            "Thought-terminating-cliches": "Using clichés to stop critical thinking",
            "Vagueness": "Being intentionally unclear"
        }
        
        prompt = f"""You are an expert in identifying logical fallacies and propaganda techniques in text.

STAGE: {stage_name}

Your task is to identify which fallacy types from the list below are present in the given text.
ONLY consider the fallacies listed for this stage. A text can have multiple fallacies.

FALLACY TYPES FOR THIS STAGE:
"""
        for label in stage_labels:
            prompt += f"- {label}: {label_descriptions.get(label, 'Propaganda technique')}\n"
        
        prompt += "\n"
        
        # Add few-shot examples
        if examples:
            prompt += "EXAMPLES:\n\n"
            example_posts = []
            for label in stage_labels:
                if label in examples and examples[label]:
                    example_posts.append((examples[label][0], label))
            
            import random
            if len(example_posts) > max_examples:
                random.seed(42)
                example_posts = random.sample(example_posts, max_examples)
            
            for idx, (example_text, label) in enumerate(example_posts, 1):
                prompt += f"Example {idx}:\n"
                prompt += f"Text: \"{example_text}\"\n"
                prompt += f"Contains: {label}\n\n"
        
        prompt += f"""Now analyze this text and identify which fallacies from the {stage_name} list are present:

TEXT: "{text}"

CRITICAL INSTRUCTIONS:
1. ONLY consider fallacies from the list above for this stage
2. Identify ALL applicable fallacies (can be multiple)
3. If no fallacies from this stage are found, return empty array
4. Be precise - only return labels that clearly apply

Output format: Return ONLY a JSON array of labels:
["Label1", "Label2"]

If no fallacies from this stage: []

Your response (JSON array only):"""
        
        return prompt

    def predict_stage(self, text: str, stage_labels: List[str], 
                     examples: Dict[str, List[str]], stage_name: str) -> List[str]:
        """Predict labels for a single stage."""
        prompt = self.create_stage_prompt(text, stage_labels, examples, stage_name)
        response = self._generate(prompt, max_tokens=256, temperature=0.0)
        labels = self._extract_labels_from_response(response, stage_labels)
        return labels

    def predict_three_stages(self, text: str, post_id: str,
                            stage1_examples: Dict[str, List[str]],
                            verbose: bool = True) -> List[str]:
        
        all_predicted_labels = []
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"Post ID: {post_id}")
            print(f"Text: {text[:200]}...")
        
        # Stage 1: High-frequency labels
        if verbose:
            print(f"\n  STAGE 1: Stage-1 fallacies")
        stage1_labels = self.predict_stage(
            text, self.STAGE_1, stage1_examples, "Stage-1"
        )
        if verbose:
            print(f"    Predicted: {stage1_labels}")
        all_predicted_labels.extend(stage1_labels)
        
        # Remove duplicates and sort
        final_labels = sorted(list(set(all_predicted_labels)))
        
        if verbose:
            print(f"\n  FINAL COMBINED: {final_labels}")
            print(f"{'='*60}")
        
        return final_labels

    def predict_batch(self, df: pd.DataFrame, 
                     stage1_examples: Dict[str, List[str]]) -> pd.DataFrame:
        predictions = []
        
        for idx, row in df.iterrows():
            print(f"\n{'#'*60}")
            print(f"Processing post {idx + 1}/{len(df)}")
            print(f"{'#'*60}")
            
            text = row['post_text']
            post_id = row['post_id']
            
            predicted_labels = self.predict_three_stages(
                text, post_id, stage1_examples,  verbose=True
            )
            
            labels_str = "|".join(predicted_labels) if predicted_labels else ""
            
            predictions.append({
                'post_id': post_id,
                'post_date': row.get('post_date', ''),
                'post_topic_keywords': row.get('post_topic_keywords', ''),
                'post_text': text,
                'labels_a1': labels_str,
                'labels_a2': labels_str
            })
        
        return pd.DataFrame(predictions)

    def save_predictions(self, predictions_df: pd.DataFrame, output_path: str):
        """Save predictions in TSV format."""
        predictions_df.to_csv(output_path, sep='\t', index=False)
        print(f"\n{'='*60}")
        print(f"Predictions saved to: {output_path}")
        print(f"{'='*60}")

    def analyze_label_distribution(self, df: pd.DataFrame, labels_col: str = "labels_a1"):
        """Analyze and print label distribution."""
        all_labels = []
        for labels_str in df[labels_col]:
            labels = str(labels_str).split('|')
            all_labels.extend([l.strip() for l in labels if l.strip()])
        
        label_counts = Counter(all_labels)
        
        print("\n" + "="*60)
        print("LABEL DISTRIBUTION")
        print("="*60)
        
        print("\nStage-1 Labels:")
        for label in self.STAGE_1:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        
        print(f"\nTotal labels: {len(all_labels)}")
        print(f"Unique labels: {len(label_counts)}")


if __name__ == "__main__":
    TRAIN_PATH = "faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-a/train-dev.tsv"
    TEST_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-a/test.tsv"
    PRED_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-a/A_1.tsv"
    
    print("="*60)
    print("THREE-STAGE MULTI-LABEL FALLACY CLASSIFIER")
    print("="*60)
    
    # Load training data
    print("\nLoading training data...")
    train_df = pd.read_csv(TRAIN_PATH, sep='\t')
    print(f"Training posts: {len(train_df)}")
    
    print("\nLoading test data...")
    test_df = pd.read_csv(TEST_PATH, sep='\t')
    print(f"Test posts: {len(test_df)}")
    

    print("\nInitializing 3-stage classifier...")
    classifier = OneStageClassifier(use_4bit=False)
    
    print("\n" + "="*60)
    print("TRAINING SET LABEL DISTRIBUTION")
    classifier.analyze_label_distribution(train_df)
    
    print("\n" + "="*60)
    print("BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA")
    print("="*60)
    
    print("\nStage 1: Stage-1 Labels")
    stage1_examples = classifier.build_few_shot_examples(
        train_df, 
        classifier.STAGE_1,
        n_examples_per_label=3
    )
    
    print("\n" + "="*60)
    print("MAKING 3-STAGE PREDICTIONS ON TEST DATA")
    print("="*60)
    
    predictions_df = classifier.predict_batch(
        test_df, stage1_examples
    )
    
    # Save predictions
    classifier.save_predictions(predictions_df, PRED_PATH)
    
    print("\n" + "="*60)
    print("PREDICTION COMPLETE")
    print("="*60)
    print(f"\nTotal predictions made: {len(predictions_df)}")
    print(f"Predictions saved to: {PRED_PATH}")
    print("="*60)

/home/ubuntu/environments/fadeit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


THREE-STAGE MULTI-LABEL FALLACY CLASSIFIER

Loading training data...
Training posts: 1152

Loading test data...
Test posts: 288

Initializing 3-stage classifier...
Loading model on device: cuda


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.23s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



TRAINING SET LABEL DISTRIBUTION

LABEL DISTRIBUTION

Stage-1 Labels:
  Appeal-to-emotion: 533
  Loaded-language: 509
  Vagueness: 341
  Name-calling-or-labelling: 308
  Doubt: 178
  Hasty-generalization: 176
  Evading-the-burden-of-proof: 140
  Flag-waving: 123
  Ad-hominem: 119
  False-analogy: 114
  Slogan: 111
  Thought-terminating-cliches: 109
  Red-herring: 99
  Appeal-to-authority: 89
  Slippery-slope: 69
  Causal-oversimplification: 64
  Strawman: 53
  Cherry-picking: 41
  False-dilemma: 30
  Circular-reasoning: 10

Total labels: 3366
Unique labels: 21

BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA

Stage 1: Stage-1 Labels

  Built examples for 20 labels in this stage:
    Appeal-to-emotion: 3 examples
    Loaded-language: 3 examples
    Vagueness: 3 examples
    Name-calling-or-labelling: 3 examples
    Doubt: 3 examples
    Hasty-generalization: 3 examples
    Evading-the-burden-of-proof: 3 examples
    Flag-waving: 3 examples
    Ad-hominem: 3 examples
    Fal

### Three stage

In [1]:
import sys
import json
import pandas as pd
from collections import defaultdict, Counter
from typing import List, Dict, Set, Tuple
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

class ThreeStageClassifier:
    
    STAGE_1 = [
        "Appeal-to-emotion", "Loaded-language", "Vagueness", 
        "Name-calling-or-labelling", "Doubt", "Hasty-generalization", 
        "Evading-the-burden-of-proof"
    ]
    
    STAGE_2 = [
        "Flag-waving", "Ad-hominem", "False-analogy", 
        "Slogan", "Thought-terminating-cliches", "Red-herring", 
        "Appeal-to-authority"
    ]
    
    STAGE_3 = [
        "Slippery-slope", "Causal-oversimplification", "Strawman", 
        "Cherry-picking", "False-dilemma", "Circular-reasoning"
    ]
    
    ALL_LABELS = STAGE_1 + STAGE_2 + STAGE_3

    def __init__(self, model_name="meta-llama/Meta-Llama-3.1-8B-Instruct", use_cpu=False, use_4bit=False):
        self.device = "cuda" if torch.cuda.is_available() and not use_cpu else "cpu"
        print(f"Loading model on device: {self.device}")
    
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
    
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def _format_for_model(self, prompt: str) -> str:
        messages = [{"role": "user", "content": prompt}]
        if hasattr(self.tokenizer, "apply_chat_template"):
            try:
                return self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            except Exception:
                return prompt
        return prompt

    def _generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.0) -> str:
        text = self._format_for_model(prompt)
        inputs = self.tokenizer(text, return_tensors="pt", padding=True).to(self.device)
        
        with torch.inference_mode():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=0.95,
                do_sample=True if temperature > 0 else False,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id or self.tokenizer.eos_token_id,
            )
        
        generated = outputs[0][inputs["input_ids"].shape[1]:]
        response = self.tokenizer.decode(generated, skip_special_tokens=True)
        
        del inputs, outputs, generated
        torch.cuda.empty_cache()
        
        return response

    def _extract_labels_from_response(self, response: str, valid_labels: List[str]) -> List[str]:
        labels = []
        
        # Strategy 1: Try JSON format
        try:
            start = response.find("{")
            end = response.rfind("}") + 1
            if start != -1 and end > start:
                json_str = response[start:end]
                data = json.loads(json_str)
                if "labels" in data:
                    labels = data["labels"]
                elif "fallacies" in data:
                    labels = data["fallacies"]
        except:
            pass
        
        # Strategy 2: Try JSON array
        if not labels:
            try:
                start = response.find("[")
                end = response.rfind("]") + 1
                if start != -1 and end > start:
                    json_str = response[start:end]
                    labels = json.loads(json_str)
            except:
                pass
        
        # Strategy 3: Extract labels mentioned in text
        if not labels:
            for label in valid_labels:
                if label in response:
                    labels.append(label)
        
        # Clean and validate labels
        valid_output = []
        for label in labels:
            label = label.strip()
            if label in valid_labels:
                valid_output.append(label)
            else:
                normalized = label.replace(" ", "-")
                if normalized in valid_labels:
                    valid_output.append(normalized)
        
        return list(set(valid_output))

    def build_few_shot_examples(self, train_df: pd.DataFrame, 
                                 labels_subset: List[str],
                                 n_examples_per_label: int = 3,
                                 labels_col: str = "labels_a1",
                                 max_text_length: int = 300) -> Dict[str, List[str]]:
        examples = defaultdict(list)
        
        train_df_shuffled = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        for idx, row in train_df_shuffled.iterrows():
            labels = str(row[labels_col]).split('|')
            labels = [l.strip() for l in labels if l.strip() and l.strip() in labels_subset]
            
            if not labels:
                continue
            
            text = row['post_text']
            if len(text) > max_text_length:
                text = text[:max_text_length] + "..."
            
            for label in labels:
                if label in labels_subset and len(examples[label]) < n_examples_per_label:
                    examples[label].append(text)
            
            if all(len(examples[label]) >= n_examples_per_label for label in labels_subset):
                break
        
        print(f"\n  Built examples for {len(examples)} labels in this stage:")
        for label in labels_subset:
            count = len(examples.get(label, []))
            print(f"    {label}: {count} examples")
        
        return dict(examples)

    def create_stage_prompt(self, text: str, stage_labels: List[str], 
                           examples: Dict[str, List[str]], 
                           stage_name: str,
                           max_examples: int = 3) -> str:
        
        label_descriptions = {
            "Ad-hominem": "Attacking the person instead of the argument",
            "Appeal-to-authority": "Using authority as evidence",
            "Appeal-to-emotion": "Manipulating emotions instead of using logic",
            "Causal-oversimplification": "Oversimplifying cause-effect relationships",
            "Cherry-picking": "Selecting only favorable evidence",
            "Circular-reasoning": "Conclusion is assumed in the premise",
            "Doubt": "Questioning credibility without evidence",
            "Evading-the-burden-of-proof": "Avoiding providing evidence",
            "False-analogy": "Comparing incomparable things",
            "False-dilemma": "Presenting only two options when more exist",
            "Flag-waving": "Appealing to nationalism/patriotism",
            "Hasty-generalization": "Drawing conclusions from insufficient evidence",
            "Loaded-language": "Using emotionally charged words",
            "Name-calling-or-labelling": "Using derogatory labels",
            "Red-herring": "Introducing irrelevant information",
            "Slippery-slope": "Claiming one thing will lead to extreme consequences",
            "Slogan": "Using catchy phrases instead of arguments",
            "Strawman": "Misrepresenting opponent's argument",
            "Thought-terminating-cliches": "Using clichés to stop critical thinking",
            "Vagueness": "Being intentionally unclear"
        }
        
        prompt = f"""You are an expert in identifying logical fallacies and propaganda techniques in text.

STAGE: {stage_name}

Your task is to identify which fallacy types from the list below are present in the given text.
ONLY consider the fallacies listed for this stage. A text can have multiple fallacies.

FALLACY TYPES FOR THIS STAGE:
"""
        for label in stage_labels:
            prompt += f"- {label}: {label_descriptions.get(label, 'Propaganda technique')}\n"
        
        prompt += "\n"
        
        # Add few-shot examples
        if examples:
            prompt += "EXAMPLES:\n\n"
            example_posts = []
            for label in stage_labels:
                if label in examples and examples[label]:
                    example_posts.append((examples[label][0], label))
            
            import random
            if len(example_posts) > max_examples:
                random.seed(42)
                example_posts = random.sample(example_posts, max_examples)
            
            for idx, (example_text, label) in enumerate(example_posts, 1):
                prompt += f"Example {idx}:\n"
                prompt += f"Text: \"{example_text}\"\n"
                prompt += f"Contains: {label}\n\n"
        
        prompt += f"""Now analyze this text and identify which fallacies from the {stage_name} list are present:

TEXT: "{text}"

CRITICAL INSTRUCTIONS:
1. ONLY consider fallacies from the list above for this stage
2. Identify ALL applicable fallacies (can be multiple)
3. If no fallacies from this stage are found, return empty array
4. Be precise - only return labels that clearly apply

Output format: Return ONLY a JSON array of labels:
["Label1", "Label2"]

If no fallacies from this stage: []

Your response (JSON array only):"""
        
        return prompt

    def predict_stage(self, text: str, stage_labels: List[str], 
                     examples: Dict[str, List[str]], stage_name: str) -> List[str]:
        """Predict labels for a single stage."""
        prompt = self.create_stage_prompt(text, stage_labels, examples, stage_name)
        response = self._generate(prompt, max_tokens=256, temperature=0.0)
        labels = self._extract_labels_from_response(response, stage_labels)
        return labels

    def predict_three_stages(self, text: str, post_id: str,
                            stage1_examples: Dict[str, List[str]],
                            stage2_examples: Dict[str, List[str]],
                            stage3_examples: Dict[str, List[str]],
                            verbose: bool = True) -> List[str]:
        
        all_predicted_labels = []
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"Post ID: {post_id}")
            print(f"Text: {text[:200]}...")
        
        # Stage 1: High-frequency labels
        if verbose:
            print(f"\n  STAGE 1: Stage-1 fallacies")
        stage1_labels = self.predict_stage(
            text, self.STAGE_1, stage1_examples, "Stage-1"
        )
        if verbose:
            print(f"    Predicted: {stage1_labels}")
        all_predicted_labels.extend(stage1_labels)
        
        # Stage 2: Mid-frequency labels
        if verbose:
            print(f"\n  STAGE 2: Stage-2 fallacies")
        stage2_labels = self.predict_stage(
            text, self.STAGE_2, stage2_examples, "Stage-2"
        )
        if verbose:
            print(f"    Predicted: {stage2_labels}")
        all_predicted_labels.extend(stage2_labels)
        
        # Stage 3: Low-frequency labels
        if verbose:
            print(f"\n  STAGE 3: Stage-3 fallacies")
        stage3_labels = self.predict_stage(
            text, self.STAGE_3, stage3_examples, "Stage-3"
        )
        if verbose:
            print(f"    Predicted: {stage3_labels}")
        all_predicted_labels.extend(stage3_labels)
        
        # Remove duplicates and sort
        final_labels = sorted(list(set(all_predicted_labels)))
        
        if verbose:
            print(f"\n  FINAL COMBINED: {final_labels}")
            print(f"{'='*60}")
        
        return final_labels

    def predict_batch(self, df: pd.DataFrame, 
                     stage1_examples: Dict[str, List[str]],
                     stage2_examples: Dict[str, List[str]],
                     stage3_examples: Dict[str, List[str]]) -> pd.DataFrame:
        predictions = []
        
        for idx, row in df.iterrows():
            print(f"\n{'#'*60}")
            print(f"Processing post {idx + 1}/{len(df)}")
            print(f"{'#'*60}")
            
            text = row['post_text']
            post_id = row['post_id']
            
            predicted_labels = self.predict_three_stages(
                text, post_id, stage1_examples, stage2_examples, stage3_examples, verbose=True
            )
            
            labels_str = "|".join(predicted_labels) if predicted_labels else ""
            
            predictions.append({
                'post_id': post_id,
                'post_date': row.get('post_date', ''),
                'post_topic_keywords': row.get('post_topic_keywords', ''),
                'post_text': text,
                'labels_a1': labels_str,
                'labels_a2': labels_str
            })
        
        return pd.DataFrame(predictions)

    def save_predictions(self, predictions_df: pd.DataFrame, output_path: str):
        """Save predictions in TSV format."""
        predictions_df.to_csv(output_path, sep='\t', index=False)
        print(f"\n{'='*60}")
        print(f"Predictions saved to: {output_path}")
        print(f"{'='*60}")

    def analyze_label_distribution(self, df: pd.DataFrame, labels_col: str = "labels_a1"):
        """Analyze and print label distribution."""
        all_labels = []
        for labels_str in df[labels_col]:
            labels = str(labels_str).split('|')
            all_labels.extend([l.strip() for l in labels if l.strip()])
        
        label_counts = Counter(all_labels)
        
        print("\n" + "="*60)
        print("LABEL DISTRIBUTION")
        print("="*60)
        
        print("\nStage-1 Labels:")
        for label in self.STAGE_1:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print("\nStage-2 Labels:")
        for label in self.STAGE_2:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print("\nStage-3 Labels:")
        for label in self.STAGE_3:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print(f"\nTotal labels: {len(all_labels)}")
        print(f"Unique labels: {len(label_counts)}")


if __name__ == "__main__":
    TRAIN_PATH = "faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-a/train-dev.tsv"
    TEST_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-a/test.tsv"
    PRED_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-a/A_1.tsv"
    
    print("="*60)
    print("THREE-STAGE MULTI-LABEL FALLACY CLASSIFIER")
    print("="*60)
    
    # Load training data
    print("\nLoading training data...")
    train_df = pd.read_csv(TRAIN_PATH, sep='\t')
    print(f"Training posts: {len(train_df)}")
    
    print("\nLoading test data...")
    test_df = pd.read_csv(TEST_PATH, sep='\t')
    print(f"Test posts: {len(test_df)}")
    

    print("\nInitializing 3-stage classifier...")
    classifier = ThreeStageClassifier(use_4bit=False)
    
    print("\n" + "="*60)
    print("TRAINING SET LABEL DISTRIBUTION")
    classifier.analyze_label_distribution(train_df)
    
    print("\n" + "="*60)
    print("BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA")
    print("="*60)
    
    print("\nStage 1: Stage-1 Labels")
    stage1_examples = classifier.build_few_shot_examples(
        train_df, 
        classifier.STAGE_1,
        n_examples_per_label=3
    )
    
    print("\nStage 2: Stage-2 Labels")
    stage2_examples = classifier.build_few_shot_examples(
        train_df,
        classifier.STAGE_2,
        n_examples_per_label=3
    )
    
    print("\nStage 3: Stage-3 Labels")
    stage3_examples = classifier.build_few_shot_examples(
        train_df,
        classifier.STAGE_3,
        n_examples_per_label=3
    )
    
    print("\n" + "="*60)
    print("MAKING 3-STAGE PREDICTIONS ON TEST DATA")
    print("="*60)
    
    predictions_df = classifier.predict_batch(
        test_df, stage1_examples, stage2_examples, stage3_examples
    )
    
    # Save predictions
    classifier.save_predictions(predictions_df, PRED_PATH)
    
    print("\n" + "="*60)
    print("PREDICTION COMPLETE")
    print("="*60)
    print(f"\nTotal predictions made: {len(predictions_df)}")
    print(f"Predictions saved to: {PRED_PATH}")
    print("="*60)

/home/ubuntu/environments/fadeit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


THREE-STAGE MULTI-LABEL FALLACY CLASSIFIER

Loading training data...
Training posts: 1152

Loading test data...
Test posts: 288

Initializing 3-stage classifier...
Loading model on device: cuda


Loading checkpoint shards: 100%|██████████| 8/8 [00:32<00:00,  4.08s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



TRAINING SET LABEL DISTRIBUTION

LABEL DISTRIBUTION

Stage-1 Labels:
  Appeal-to-emotion: 533
  Loaded-language: 509
  Vagueness: 341
  Name-calling-or-labelling: 308
  Doubt: 178
  Hasty-generalization: 176
  Evading-the-burden-of-proof: 140

Stage-2 Labels:
  Flag-waving: 123
  Ad-hominem: 119
  False-analogy: 114
  Slogan: 111
  Thought-terminating-cliches: 109
  Red-herring: 99
  Appeal-to-authority: 89

Stage-3 Labels:
  Slippery-slope: 69
  Causal-oversimplification: 64
  Strawman: 53
  Cherry-picking: 41
  False-dilemma: 30
  Circular-reasoning: 10

Total labels: 3366
Unique labels: 21

BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA

Stage 1: Stage-1 Labels

  Built examples for 7 labels in this stage:
    Appeal-to-emotion: 3 examples
    Loaded-language: 3 examples
    Vagueness: 3 examples
    Name-calling-or-labelling: 3 examples
    Doubt: 3 examples
    Hasty-generalization: 3 examples
    Evading-the-burden-of-proof: 3 examples

Stage 2: Stage-2 Labels

  B

### Four stage

In [1]:
import sys
import json
import pandas as pd
from collections import defaultdict, Counter
from typing import List, Dict, Set, Tuple
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

class ThreeStageClassifier:
    
    STAGE_1 = [
        "Appeal-to-emotion", "Loaded-language", "Vagueness", 
        "Name-calling-or-labelling"
    ]
    
    STAGE_2 = [
        "Doubt", "Hasty-generalization", 
        "Evading-the-burden-of-proof","Flag-waving"
    ]
    
    STAGE_3 = [
        "Ad-hominem", "False-analogy", "Slogan",
        "Thought-terminating-cliches", "Red-herring", 
        "Appeal-to-authority","Slippery-slope", "Causal-oversimplification"
    ]

    STAGE_4 = [
        "Strawman", "Cherry-picking", "False-dilemma",
        "Circular-reasoning"
    ]
    
    ALL_LABELS = STAGE_1 + STAGE_2 + STAGE_3 + STAGE_4

    def __init__(self, model_name="meta-llama/Meta-Llama-3.1-8B-Instruct", use_cpu=False, use_4bit=False):
        self.device = "cuda" if torch.cuda.is_available() and not use_cpu else "cpu"
        print(f"Loading model on device: {self.device}")
    
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
    
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def _format_for_model(self, prompt: str) -> str:
        messages = [{"role": "user", "content": prompt}]
        if hasattr(self.tokenizer, "apply_chat_template"):
            try:
                return self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            except Exception:
                return prompt
        return prompt

    def _generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.0) -> str:
        text = self._format_for_model(prompt)
        inputs = self.tokenizer(text, return_tensors="pt", padding=True).to(self.device)
        
        with torch.inference_mode():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=0.95,
                do_sample=True if temperature > 0 else False,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id or self.tokenizer.eos_token_id,
            )
        
        generated = outputs[0][inputs["input_ids"].shape[1]:]
        response = self.tokenizer.decode(generated, skip_special_tokens=True)
        
        del inputs, outputs, generated
        torch.cuda.empty_cache()
        
        return response

    def _extract_labels_from_response(self, response: str, valid_labels: List[str]) -> List[str]:
        labels = []
        
        # Strategy 1: Try JSON format
        try:
            start = response.find("{")
            end = response.rfind("}") + 1
            if start != -1 and end > start:
                json_str = response[start:end]
                data = json.loads(json_str)
                if "labels" in data:
                    labels = data["labels"]
                elif "fallacies" in data:
                    labels = data["fallacies"]
        except:
            pass
        
        # Strategy 2: Try JSON array
        if not labels:
            try:
                start = response.find("[")
                end = response.rfind("]") + 1
                if start != -1 and end > start:
                    json_str = response[start:end]
                    labels = json.loads(json_str)
            except:
                pass
        
        # Strategy 3: Extract labels mentioned in text
        if not labels:
            for label in valid_labels:
                if label in response:
                    labels.append(label)
        
        # Clean and validate labels
        valid_output = []
        for label in labels:
            label = label.strip()
            if label in valid_labels:
                valid_output.append(label)
            else:
                normalized = label.replace(" ", "-")
                if normalized in valid_labels:
                    valid_output.append(normalized)
        
        return list(set(valid_output))

    def build_few_shot_examples(self, train_df: pd.DataFrame, 
                                 labels_subset: List[str],
                                 n_examples_per_label: int = 3,
                                 labels_col: str = "labels_a1",
                                 max_text_length: int = 300) -> Dict[str, List[str]]:
        examples = defaultdict(list)
        
        train_df_shuffled = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        for idx, row in train_df_shuffled.iterrows():
            labels = str(row[labels_col]).split('|')
            labels = [l.strip() for l in labels if l.strip() and l.strip() in labels_subset]
            
            if not labels:
                continue
            
            text = row['post_text']
            if len(text) > max_text_length:
                text = text[:max_text_length] + "..."
            
            for label in labels:
                if label in labels_subset and len(examples[label]) < n_examples_per_label:
                    examples[label].append(text)
            
            if all(len(examples[label]) >= n_examples_per_label for label in labels_subset):
                break
        
        print(f"\n  Built examples for {len(examples)} labels in this stage:")
        for label in labels_subset:
            count = len(examples.get(label, []))
            print(f"    {label}: {count} examples")
        
        return dict(examples)

    def create_stage_prompt(self, text: str, stage_labels: List[str], 
                           examples: Dict[str, List[str]], 
                           stage_name: str,
                           max_examples: int = 3) -> str:
        
        label_descriptions = {
            "Ad-hominem": "Attacking the person instead of the argument",
            "Appeal-to-authority": "Using authority as evidence",
            "Appeal-to-emotion": "Manipulating emotions instead of using logic",
            "Causal-oversimplification": "Oversimplifying cause-effect relationships",
            "Cherry-picking": "Selecting only favorable evidence",
            "Circular-reasoning": "Conclusion is assumed in the premise",
            "Doubt": "Questioning credibility without evidence",
            "Evading-the-burden-of-proof": "Avoiding providing evidence",
            "False-analogy": "Comparing incomparable things",
            "False-dilemma": "Presenting only two options when more exist",
            "Flag-waving": "Appealing to nationalism/patriotism",
            "Hasty-generalization": "Drawing conclusions from insufficient evidence",
            "Loaded-language": "Using emotionally charged words",
            "Name-calling-or-labelling": "Using derogatory labels",
            "Red-herring": "Introducing irrelevant information",
            "Slippery-slope": "Claiming one thing will lead to extreme consequences",
            "Slogan": "Using catchy phrases instead of arguments",
            "Strawman": "Misrepresenting opponent's argument",
            "Thought-terminating-cliches": "Using clichés to stop critical thinking",
            "Vagueness": "Being intentionally unclear"
        }
        
        prompt = f"""You are an expert in identifying logical fallacies and propaganda techniques in text.

STAGE: {stage_name}

Your task is to identify which fallacy types from the list below are present in the given text.
ONLY consider the fallacies listed for this stage. A text can have multiple fallacies.

FALLACY TYPES FOR THIS STAGE:
"""
        for label in stage_labels:
            prompt += f"- {label}: {label_descriptions.get(label, 'Propaganda technique')}\n"
        
        prompt += "\n"
        
        # Add few-shot examples
        if examples:
            prompt += "EXAMPLES:\n\n"
            example_posts = []
            for label in stage_labels:
                if label in examples and examples[label]:
                    example_posts.append((examples[label][0], label))
            
            import random
            if len(example_posts) > max_examples:
                random.seed(42)
                example_posts = random.sample(example_posts, max_examples)
            
            for idx, (example_text, label) in enumerate(example_posts, 1):
                prompt += f"Example {idx}:\n"
                prompt += f"Text: \"{example_text}\"\n"
                prompt += f"Contains: {label}\n\n"
        
        prompt += f"""Now analyze this text and identify which fallacies from the {stage_name} list are present:

TEXT: "{text}"

CRITICAL INSTRUCTIONS:
1. ONLY consider fallacies from the list above for this stage
2. Identify ALL applicable fallacies (can be multiple)
3. If no fallacies from this stage are found, return empty array
4. Be precise - only return labels that clearly apply

Output format: Return ONLY a JSON array of labels:
["Label1", "Label2"]

If no fallacies from this stage: []

Your response (JSON array only):"""
        
        return prompt

    def predict_stage(self, text: str, stage_labels: List[str], 
                     examples: Dict[str, List[str]], stage_name: str) -> List[str]:
        """Predict labels for a single stage."""
        prompt = self.create_stage_prompt(text, stage_labels, examples, stage_name)
        response = self._generate(prompt, max_tokens=256, temperature=0.0)
        labels = self._extract_labels_from_response(response, stage_labels)
        return labels

    def predict_three_stages(self, text: str, post_id: str,
                            stage1_examples: Dict[str, List[str]],
                            stage2_examples: Dict[str, List[str]],
                            stage3_examples: Dict[str, List[str]],
                            stage4_examples: Dict[str, List[str]],
                            verbose: bool = True) -> List[str]:
        
        all_predicted_labels = []
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"Post ID: {post_id}")
            print(f"Text: {text[:200]}...")
        
        # Stage 1: High-frequency labels
        if verbose:
            print(f"\n  STAGE 1: Stage-1 fallacies")
        stage1_labels = self.predict_stage(
            text, self.STAGE_1, stage1_examples, "Stage-1"
        )
        if verbose:
            print(f"    Predicted: {stage1_labels}")
        all_predicted_labels.extend(stage1_labels)
        
        # Stage 2: Mid-frequency labels
        if verbose:
            print(f"\n  STAGE 2: Stage-2 fallacies")
        stage2_labels = self.predict_stage(
            text, self.STAGE_2, stage2_examples, "Stage-2"
        )
        if verbose:
            print(f"    Predicted: {stage2_labels}")
        all_predicted_labels.extend(stage2_labels)
        
        # Stage 3: Low-frequency labels
        if verbose:
            print(f"\n  STAGE 3: Stage-3 fallacies")
        stage3_labels = self.predict_stage(
            text, self.STAGE_3, stage3_examples, "Stage-3"
        )
        if verbose:
            print(f"    Predicted: {stage3_labels}")
        all_predicted_labels.extend(stage3_labels)

        # Stage 4: Lowest-frequency labels
        if verbose:
            print(f"\n  STAGE 4: Stage-4 fallacies")
        stage4_labels = self.predict_stage(
            text, self.STAGE_4, stage4_examples, "Stage-4"
        )
        if verbose:
            print(f"    Predicted: {stage4_labels}")
        all_predicted_labels.extend(stage4_labels)
        
        # Remove duplicates and sort
        final_labels = sorted(list(set(all_predicted_labels)))
        
        if verbose:
            print(f"\n  FINAL COMBINED: {final_labels}")
            print(f"{'='*60}")
        
        return final_labels

    def predict_batch(self, df: pd.DataFrame, 
                     stage1_examples: Dict[str, List[str]],
                     stage2_examples: Dict[str, List[str]],
                     stage3_examples: Dict[str, List[str]],
                     stage4_examples: Dict[str, List[str]]) -> pd.DataFrame:
        predictions = []
        
        for idx, row in df.iterrows():
            print(f"\n{'#'*60}")
            print(f"Processing post {idx + 1}/{len(df)}")
            print(f"{'#'*60}")
            
            text = row['post_text']
            post_id = row['post_id']
            
            predicted_labels = self.predict_three_stages(
                text, post_id, stage1_examples, stage2_examples, stage3_examples, stage4_examples, verbose=True
            )
            
            labels_str = "|".join(predicted_labels) if predicted_labels else ""
            
            predictions.append({
                'post_id': post_id,
                'post_date': row.get('post_date', ''),
                'post_topic_keywords': row.get('post_topic_keywords', ''),
                'post_text': text,
                'labels_a1': labels_str,
                'labels_a2': labels_str
            })
        
        return pd.DataFrame(predictions)

    def save_predictions(self, predictions_df: pd.DataFrame, output_path: str):
        """Save predictions in TSV format."""
        predictions_df.to_csv(output_path, sep='\t', index=False)
        print(f"\n{'='*60}")
        print(f"Predictions saved to: {output_path}")
        print(f"{'='*60}")

    def analyze_label_distribution(self, df: pd.DataFrame, labels_col: str = "labels_a1"):
        """Analyze and print label distribution."""
        all_labels = []
        for labels_str in df[labels_col]:
            labels = str(labels_str).split('|')
            all_labels.extend([l.strip() for l in labels if l.strip()])
        
        label_counts = Counter(all_labels)
        
        print("\n" + "="*60)
        print("LABEL DISTRIBUTION")
        print("="*60)
        
        print("\nStage-1 Labels:")
        for label in self.STAGE_1:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print("\nStage-2 Labels:")
        for label in self.STAGE_2:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print("\nStage-3 Labels:")
        for label in self.STAGE_3:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")

        print("\nStage-4 Labels:")
        for label in self.STAGE_4:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print(f"\nTotal labels: {len(all_labels)}")
        print(f"Unique labels: {len(label_counts)}")


if __name__ == "__main__":
    TRAIN_PATH = "faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-a/train-dev.tsv"
    TEST_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-a/test.tsv"
    PRED_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-a/A_3.tsv"
    
    print("="*60)
    print("THREE-STAGE MULTI-LABEL FALLACY CLASSIFIER")
    print("="*60)
    
    # Load training data
    print("\nLoading training data...")
    train_df = pd.read_csv(TRAIN_PATH, sep='\t')
    print(f"Training posts: {len(train_df)}")
    
    print("\nLoading test data...")
    test_df = pd.read_csv(TEST_PATH, sep='\t')
    print(f"Test posts: {len(test_df)}")
    

    print("\nInitializing 3-stage classifier...")
    classifier = ThreeStageClassifier(use_4bit=False)
    
    print("\n" + "="*60)
    print("TRAINING SET LABEL DISTRIBUTION")
    classifier.analyze_label_distribution(train_df)
    
    print("\n" + "="*60)
    print("BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA")
    print("="*60)
    
    print("\nStage 1: Stage-1 Labels")
    stage1_examples = classifier.build_few_shot_examples(
        train_df, 
        classifier.STAGE_1,
        n_examples_per_label=3
    )
    
    print("\nStage 2: Stage-2 Labels")
    stage2_examples = classifier.build_few_shot_examples(
        train_df,
        classifier.STAGE_2,
        n_examples_per_label=3
    )
    
    print("\nStage 3: Stage-3 Labels")
    stage3_examples = classifier.build_few_shot_examples(
        train_df,
        classifier.STAGE_3,
        n_examples_per_label=3
    )

    print("\nStage 4: Stage-4 Labels")
    stage4_examples = classifier.build_few_shot_examples(
        train_df,
        classifier.STAGE_4,
        n_examples_per_label=3
    )
    
    print("\n" + "="*60)
    print("MAKING 3-STAGE PREDICTIONS ON TEST DATA")
    print("="*60)
    
    predictions_df = classifier.predict_batch(
        test_df, stage1_examples, stage2_examples, stage3_examples, stage4_examples
    )
    
    # Save predictions
    classifier.save_predictions(predictions_df, PRED_PATH)
    
    print("\n" + "="*60)
    print("PREDICTION COMPLETE")
    print("="*60)
    print(f"\nTotal predictions made: {len(predictions_df)}")
    print(f"Predictions saved to: {PRED_PATH}")
    print("="*60)

/home/ubuntu/environments/fadeit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


THREE-STAGE MULTI-LABEL FALLACY CLASSIFIER

Loading training data...
Training posts: 1152

Loading test data...
Test posts: 288

Initializing 3-stage classifier...
Loading model on device: cuda


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.31it/s]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



TRAINING SET LABEL DISTRIBUTION

LABEL DISTRIBUTION

Stage-1 Labels:
  Appeal-to-emotion: 533
  Loaded-language: 509
  Vagueness: 341
  Name-calling-or-labelling: 308

Stage-2 Labels:
  Doubt: 178
  Hasty-generalization: 176
  Evading-the-burden-of-proof: 140
  Flag-waving: 123

Stage-3 Labels:
  Ad-hominem: 119
  False-analogy: 114
  Slogan: 111
  Thought-terminating-cliches: 109
  Red-herring: 99
  Appeal-to-authority: 89
  Slippery-slope: 69
  Causal-oversimplification: 64

Stage-4 Labels:
  Strawman: 53
  Cherry-picking: 41
  False-dilemma: 30
  Circular-reasoning: 10

Total labels: 3366
Unique labels: 21

BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA

Stage 1: Stage-1 Labels

  Built examples for 4 labels in this stage:
    Appeal-to-emotion: 3 examples
    Loaded-language: 3 examples
    Vagueness: 3 examples
    Name-calling-or-labelling: 3 examples

Stage 2: Stage-2 Labels

  Built examples for 4 labels in this stage:
    Doubt: 3 examples
    Hasty-generalizati

# CONLL

### Single stage

In [1]:
import sys
import json
import re
import os
from collections import defaultdict, Counter
from typing import List, Dict, Set
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


class ThreeStageSpanExtractor:
    
    STAGE_1 = [
        "Appeal-to-emotion",      
        "Vagueness",             
        "Doubt",                  
        "Evading-the-burden-of-proof",  
        "False-analogy",          
        "Hasty-generalization",   
        "Strawman",               
        "Ad-hominem",             
        "Loaded-language",        
        "Red-herring",            
        "Causal-oversimplification",  
        "Cherry-picking",         
        "Slippery-slope",         
        "Name-calling-or-labelling",  
        "Flag-waving",            
        "False-dilemma",          
        "Thought-terminating-cliches",  
        "Appeal-to-authority",    
        "Slogan",                 
        "Circular-reasoning",     
    ]
    
    ALL_LABELS = STAGE_1 

    def __init__(self, model_name="meta-llama/Meta-Llama-3.1-8B-Instruct", use_cpu=False, use_4bit=False):
        self.device = "cuda" if torch.cuda.is_available() and not use_cpu else "cpu"
        print(f"Loading model on device: {self.device}")
    
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
    
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def parse_conll_file(self, filepath: str) -> List[Dict]:
        posts = []
        current_post = None
        
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.rstrip('\n')
                
                if line.startswith("# post_id = "):
                    if current_post:
                        posts.append(current_post)
                    current_post = {
                        'post_id': line.split(" = ")[1],
                        'date': '',
                        'topic': '',
                        'text': '',
                        'tokens': [],
                        'bio_tags_a1': [],
                        'bio_tags_a2': []
                    }
                elif line.startswith("# post_date = "):
                    current_post['date'] = line.split(" = ")[1]
                elif line.startswith("# post_topic_keywords = "):
                    current_post['topic'] = line.split(" = ")[1]
                elif line.startswith("# post_text = "):
                    current_post['text'] = line.split(" = ")[1]
                elif len(line.strip()) == 0:
                    if current_post and current_post['tokens']:
                        current_post['spans_a1'] = self._bio_to_spans(
                            current_post['tokens'], current_post['bio_tags_a1']
                        )
                        current_post['spans_a2'] = self._bio_to_spans(
                            current_post['tokens'], current_post['bio_tags_a2']
                        )
                        posts.append(current_post)
                        current_post = None
                elif current_post is not None:
                    parts = line.split('\t')
                    if len(parts) >= 4:
                        current_post['tokens'].append(parts[1])
                        current_post['bio_tags_a1'].append(parts[2])
                        current_post['bio_tags_a2'].append(parts[3])
        
        if current_post and current_post['tokens']:
            current_post['spans_a1'] = self._bio_to_spans(
                current_post['tokens'], current_post['bio_tags_a1']
            )
            current_post['spans_a2'] = self._bio_to_spans(
                current_post['tokens'], current_post['bio_tags_a2']
            )
            posts.append(current_post)
        
        print(f"Parsed {len(posts)} posts from {filepath}")
        return posts

    def _bio_to_spans(self, tokens: List[str], bio_tags: List[str]) -> List[Dict]:
        active_spans = {}
        completed_spans = []
        
        for idx, (token, bio_tag_str) in enumerate(zip(tokens, bio_tags)):
            bio_tags_list = bio_tag_str.split('|')
            current_labels = set()
            
            for bio_tag in bio_tags_list:
                bio_tag = bio_tag.strip()
                if bio_tag.startswith("B-"):
                    label = bio_tag[2:]
                    current_labels.add(label)
                    
                    if label in active_spans:
                        span = active_spans[label]
                        completed_spans.append({
                            'label': label,
                            'start_tok': span['start_tok'],
                            'end_tok': idx - 1,
                            'text': " ".join(span['tokens'])
                        })
                    
                    active_spans[label] = {'start_tok': idx, 'tokens': [token]}
                    
                elif bio_tag.startswith("I-"):
                    label = bio_tag[2:]
                    current_labels.add(label)
                    if label in active_spans:
                        active_spans[label]['tokens'].append(token)
                    else:
                        active_spans[label] = {'start_tok': idx, 'tokens': [token]}
            
            labels_to_close = set(active_spans.keys()) - current_labels
            for label in labels_to_close:
                span = active_spans[label]
                completed_spans.append({
                    'label': label,
                    'start_tok': span['start_tok'],
                    'end_tok': idx - 1,
                    'text': " ".join(span['tokens'])
                })
                del active_spans[label]
        
        for label, span in active_spans.items():
            completed_spans.append({
                'label': label,
                'start_tok': span['start_tok'],
                'end_tok': len(tokens) - 1,
                'text': " ".join(span['tokens'])
            })
        
        return completed_spans

    def _format_for_model(self, prompt: str) -> str:
        messages = [{"role": "user", "content": prompt}]
        if hasattr(self.tokenizer, "apply_chat_template"):
            try:
                return self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            except Exception:
                return prompt
        return prompt

    def _generate(self, prompt: str, max_tokens: int = 1024, temperature: float = 0.0) -> str:
        text = self._format_for_model(prompt)
        inputs = self.tokenizer(text, return_tensors="pt", padding=True).to(self.device)
        
        with torch.inference_mode():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=0.95,
                do_sample=True if temperature > 0 else False,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id or self.tokenizer.eos_token_id,
            )
        
        generated = outputs[0][inputs["input_ids"].shape[1]:]
        response = self.tokenizer.decode(generated, skip_special_tokens=True)
        
        del inputs, outputs, generated
        torch.cuda.empty_cache()
        
        return response

    def _extract_spans_from_response(self, response: str, valid_labels: List[str]) -> List[Dict]:
        spans = []
        
        # Clean response
        response = re.sub(r'```json\s*', '', response)
        response = re.sub(r'```\s*', '', response)
        response = response.strip()
        
        # Try JSON array
        try:
            start = response.find("[")
            end = response.rfind("]") + 1
            if start != -1 and end > start:
                json_str = response[start:end]
                data = json.loads(json_str)
                if isinstance(data, list):
                    spans = data
        except:
            pass
        
        # Try JSON object
        if not spans:
            try:
                start = response.find("{")
                end = response.rfind("}") + 1
                if start != -1 and end > start:
                    json_str = response[start:end]
                    data = json.loads(json_str)
                    if "spans" in data:
                        spans = data["spans"]
            except:
                pass
        
        # Line-by-line extraction
        if not spans:
            for line in response.split('\n'):
                text_match = re.search(r'"text"\s*:\s*"([^"]+)"', line)
                label_match = re.search(r'"label"\s*:\s*"([^"]+)"', line)
                if text_match and label_match:
                    spans.append({
                        "text": text_match.group(1),
                        "label": label_match.group(1)
                    })
        
        # Validate and filter spans
        valid_spans = []
        for span in spans:
            if not isinstance(span, dict):
                continue
            
            text = span.get("text", "").strip()
            label = span.get("label", "").strip().replace(" ", "-").replace("_", "-")
            
            if not text or not label:
                continue
            
            # Only accept labels from current stage
            if label in valid_labels:
                valid_spans.append({"text": text, "label": label})
            else:
                for valid_label in valid_labels:
                    if valid_label.lower() == label.lower():
                        valid_spans.append({"text": text, "label": valid_label})
                        break
        
        return valid_spans

    def build_few_shot_examples(self, posts: List[Dict], 
                                labels_subset: List[str],
                                n_examples_per_label: int = 3,
                                annotator: str = "a1") -> Dict[str, List[str]]:
        examples = defaultdict(list)
        spans_key = f'spans_{annotator}'
        
        # Collect spans per label
        label_spans = defaultdict(list)
        for post in posts:
            for span in post.get(spans_key, []):
                label = span['label']
                text = span['text'].strip()
                if label in labels_subset and len(text.split()) >= 2:
                    label_spans[label].append(text)
        
        # Select diverse examples
        for label, span_list in label_spans.items():
            unique_spans = list(set(span_list))
            sorted_spans = sorted(unique_spans, key=len)
            
            selected = []
            if len(sorted_spans) <= n_examples_per_label:
                selected = sorted_spans
            else:
                step = len(sorted_spans) // n_examples_per_label
                for i in range(n_examples_per_label):
                    idx = min(i * step, len(sorted_spans) - 1)
                    selected.append(sorted_spans[idx])
            
            examples[label] = selected
        
        print(f"\n  Built examples for {len(examples)} labels in this tier:")
        for label in labels_subset:
            count = len(examples.get(label, []))
            print(f"    {label}: {count} examples")
        
        return dict(examples)

    def create_stage_prompt(self, text: str, stage_labels: List[str],
                               examples: Dict[str, List[str]],
                               stage_name: str,
                               max_examples_per_label: int = 3) -> str:

        label_descriptions = {
            "Ad-hominem": "Attacking the speaker instead of the argument",
            "Appeal-to-authority": "Claiming something is true just because an authority says so",
            "Appeal-to-emotion": "Manipulating feelings instead of providing logic",
            "Causal-oversimplification": "Reducing complex cause-effect relationships to a simple claim",
            "Cherry-picking": "Mentioning only evidence that supports a claim",
            "Circular-reasoning": "The conclusion is also used as a premise",
            "Doubt": "Casting suspicion without offering evidence",
            "Evading-the-burden-of-proof": "Making claims without backing them up",
            "False-analogy": "Comparing two unrelated things as if they are similar",
            "False-dilemma": "Claiming only two options exist when more do",
            "Flag-waving": "Appealing to patriotism or national identity",
            "Hasty-generalization": "Concluding based on very limited data",
            "Loaded-language": "Using emotionally charged language",
            "Name-calling-or-labelling": "Using insult or label instead of argument",
            "Red-herring": "Bringing up irrelevant information to distract",
            "Slippery-slope": "Claiming one event will definitely lead to a dramatic outcome",
            "Slogan": "Repeating a catchy phrase instead of reasoning",
            "Strawman": "Distorting someone's argument to attack it more easily",
            "Thought-terminating-cliches": "Phrases that shut down critical thinking",
            "Vagueness": "Statements that are unclear and undefined"

        }

        prompt = f"""You are an expert in identifying logical fallacies in text.

    STAGE: {stage_name}

    Target Fallacies:
    """
        for label in stage_labels:
            prompt += f"- {label}: {label_descriptions.get(label, 'Propaganda technique')}\n"

        if examples:
            prompt += "\nExamples:\n"
            for label in stage_labels:
                if label in examples and examples[label]:
                    prompt += f"\n{label}:\n"
                    for ex in examples[label][:max_examples_per_label]:
                        prompt += f'  - "{ex}"\n'

        prompt += f"""
    TEXT TO ANALYZE:
    "{text}"

    ANALYSIS PROCESS:
    1. First, read the entire text carefully
    2. For each sentence/phrase, ask: "Does this match any of the {len(stage_labels)} target fallacies?"
    3. If yes, identify which specific fallacy and extract the exact span
    4. Ignore any fallacies NOT in the target list above
    5. Compile all matches

    Think through your analysis step by step, then provide the final JSON output:

    {{"reasoning": "brief explanation of what you found", "spans": [{{"text": "exact span", "label": "Label-name"}}]}}

    Your response:"""

        return prompt


    def predict_stage(self, text: str, stage_labels: List[str],
                     examples: Dict[str, List[str]], stage_name: str) -> List[Dict]:
        prompt = self.create_stage_prompt(text, stage_labels, examples, stage_name)
        response = self._generate(prompt, max_tokens=1024, temperature=0.0)
        spans = self._extract_spans_from_response(response, stage_labels)
        return spans

    def predict_three_stages(self, text: str, post_id: str,
                            stage1_examples: Dict[str, List[str]],
                            verbose: bool = True) -> List[Dict]:
        
        all_spans = []
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"Post ID: {post_id}")
            print(f"Text: {text[:200]}...")
        
        # Stage 1: High-frequency labels
        if verbose:
            print(f"\n  STAGE 1: High-frequency fallacies")
        stage1_spans = self.predict_stage(
            text, self.STAGE_1, stage1_examples, "High-Frequency"
        )
        if verbose:
            print(f"    Found {len(stage1_spans)} spans")
            for span in stage1_spans[:3]:
                print(f"      [{span['label']}] \"{span['text'][:50]}...\"")
        all_spans.extend(stage1_spans)
        
        if verbose:
            print(f"\n  TOTAL: {len(all_spans)} spans across all stages")
            print(f"{'='*60}")
        
        return all_spans

    def spans_to_bio(self, text: str, tokens: List[str], spans: List[Dict]) -> List[str]:
        token_labels = [set() for _ in tokens]
        reconstructed = " ".join(tokens)
        reconstructed_lower = reconstructed.lower()
        
        for span in spans:
            span_text = span['text'].strip()
            label = span['label']
            span_text_lower = span_text.lower()
            
            span_start = reconstructed_lower.find(span_text_lower)
            
            if span_start != -1:
                prefix = reconstructed[:span_start]
                tokens_before = len(prefix.split()) if prefix.strip() else 0
                span_token_count = len(span_text.split())
                
                start_idx = tokens_before
                end_idx = min(start_idx + span_token_count, len(tokens))
                
                if start_idx < len(tokens):
                    token_labels[start_idx].add(f"B-{label}")
                    for idx in range(start_idx + 1, end_idx):
                        token_labels[idx].add(f"I-{label}")
                    continue
            
            # Fuzzy matching fallback
            span_tokens = span_text_lower.split()
            if not span_tokens:
                continue
            
            tokens_lower = [t.lower() for t in tokens]
            best_match_start = -1
            best_match_score = 0.0
            
            for i in range(len(tokens_lower) - len(span_tokens) + 1):
                window = tokens_lower[i:i + len(span_tokens)]
                matches = sum(1 for w, s in zip(window, span_tokens) if w == s or w in s or s in w)
                score = matches / len(span_tokens)
                
                if score > best_match_score:
                    best_match_score = score
                    best_match_start = i
            
            if best_match_start >= 0 and best_match_score >= 0.5:
                end_idx = best_match_start + len(span_tokens)
                token_labels[best_match_start].add(f"B-{label}")
                for idx in range(best_match_start + 1, end_idx):
                    token_labels[idx].add(f"I-{label}")
        
        # Convert to BIO format
        bio_tags = []
        for label_set in token_labels:
            if not label_set:
                bio_tags.append("O")
            else:
                bio_tags.append("|".join(sorted(label_set)))
        
        # Fix BIO sequence
        bio_tags = self._fix_bio_sequence(bio_tags)
        
        return bio_tags

    def _fix_bio_sequence(self, bio_tags: List[str]) -> List[str]:
        fixed_tags = []
        
        for idx, tag in enumerate(bio_tags):
            if tag == "O":
                fixed_tags.append(tag)
                continue
            
            labels_with_b = set()
            labels_with_i = set()
            
            for single_tag in tag.split("|"):
                if single_tag.startswith("B-"):
                    labels_with_b.add(single_tag[2:])
                elif single_tag.startswith("I-"):
                    labels_with_i.add(single_tag[2:])
            
            final_labels = []
            
            # Add B- tags
            for label in sorted(labels_with_b):
                final_labels.append(f"B-{label}")
            
            # Process I- tags
            for label in sorted(labels_with_i):
                if label in labels_with_b:
                    continue
                
                if idx > 0:
                    prev_tag = fixed_tags[idx - 1]
                    if f"B-{label}" in prev_tag or f"I-{label}" in prev_tag:
                        final_labels.append(f"I-{label}")
                    else:
                        final_labels.append(f"B-{label}")
                else:
                    final_labels.append(f"B-{label}")
            
            if final_labels:
                fixed_tags.append("|".join(sorted(final_labels)))
            else:
                fixed_tags.append("O")
        
        return fixed_tags

    def _load_checkpoint(self, checkpoint_path: str) -> Set[str]:
        if os.path.exists(checkpoint_path):
            try:
                with open(checkpoint_path, 'r', encoding='utf-8') as f:
                    processed_ids = set(json.load(f))
                print(f"\nCheckpoint found: {len(processed_ids)} posts already processed")
                return processed_ids
            except:
                print("\nCheckpoint file corrupted, starting fresh")
                return set()
        return set()

    def _save_checkpoint(self, checkpoint_path: str, processed_ids: Set[str]):
        with open(checkpoint_path, 'w', encoding='utf-8') as f:
            json.dump(list(processed_ids), f)

    def write_conll_predictions(self, posts: List[Dict], output_path: str,
                               stage1_examples: Dict[str, List[str]]):        
        checkpoint_path = output_path + ".checkpoint"
        processed_ids = self._load_checkpoint(checkpoint_path)
        
        # Determine write mode
        write_mode = 'a' if processed_ids else 'w'
        
        with open(output_path, write_mode, encoding='utf-8') as f:
            for idx, post in enumerate(posts):
                # Skip if already processed
                if post['post_id'] in processed_ids:
                    print(f"\nSkipping post {idx+1}/{len(posts)}: {post['post_id']} (already processed)")
                    continue
                
                print(f"\n{'#'*60}")
                print(f"Processing post {idx+1}/{len(posts)}: {post['post_id']}")
                print(f"{'#'*60}")
                
                # Write metadata
                f.write(f"# post_id = {post['post_id']}\n")
                f.write(f"# post_date = {post['date']}\n")
                f.write(f"# post_topic_keywords = {post['topic']}\n")
                f.write(f"# post_text = {post['text']}\n")
                
                # 3-stage prediction
                predicted_spans = self.predict_three_stages(
                    post['text'], post['post_id'],
                    stage1_examples, verbose=True
                )
                
                # Convert to BIO
                bio_tags = self.spans_to_bio(post['text'], post['tokens'], predicted_spans)
                
                # Ensure correct length
                if len(bio_tags) != len(post['tokens']):
                    print(f"   WARNING: Length mismatch! Fixing...")
                    if len(bio_tags) < len(post['tokens']):
                        bio_tags.extend(['O'] * (len(post['tokens']) - len(bio_tags)))
                    else:
                        bio_tags = bio_tags[:len(post['tokens'])]
                
                # Write tokens
                for tok_idx, (token, bio_tag) in enumerate(zip(post['tokens'], bio_tags), start=1):
                    f.write(f"{tok_idx}\t{token}\t{bio_tag}\t{bio_tag}\n")
                f.write("\n")
                
                # Flush to disk immediately
                f.flush()
                
                # Update checkpoint
                processed_ids.add(post['post_id'])
                self._save_checkpoint(checkpoint_path, processed_ids)
        
        # Clean up checkpoint file on successful completion
        if len(processed_ids) == len(posts):
            if os.path.exists(checkpoint_path):
                os.remove(checkpoint_path)
                print(f"\nCheckpoint file removed (all posts processed)")
        
        print(f"\n{'='*60}")
        print(f"Predictions saved to: {output_path}")
        print(f"{'='*60}")

    def analyze_label_distribution(self, posts: List[Dict], annotator: str = "a1"):
        spans_key = f'spans_{annotator}'
        label_counts = Counter()
        
        for post in posts:
            for span in post.get(spans_key, []):
                label_counts[span['label']] += 1
        
        print("\n" + "="*60)
        print("LABEL DISTRIBUTION IN SPANS")
        print("="*60)
        
        print("\nHigh-Frequency Labels:")
        for label in self.STAGE_1:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
                
        print(f"\nTotal spans: {sum(label_counts.values())}")


if __name__ == "__main__":
    TRAIN_CONLL_PATH = "faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-b/train-dev.conll"
    TEST_CONLL_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-b/test.conll"
    PRED_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-b/B_1.conll"
    
    print("="*60)
    print("THREE-STAGE SPAN EXTRACTION")
    print("="*60)
    
    print("\nInitializing span extractor...")
    extractor = ThreeStageSpanExtractor(use_4bit=False)
    
    # Load training data
    print("\nParsing training CoNLL file...")
    train_posts = extractor.parse_conll_file(TRAIN_CONLL_PATH)
    print(f"Training posts: {len(train_posts)}")
    
    # Load test data (without ground truth labels)
    print("\nParsing test CoNLL file...")
    test_posts = extractor.parse_conll_file(TEST_CONLL_PATH)
    print(f"Test posts: {len(test_posts)}")
    
    # Analyze training set distribution
    print("\n" + "="*60)
    print("TRAINING SET LABEL DISTRIBUTION")
    print("="*60)
    extractor.analyze_label_distribution(train_posts)
    
    print("\n" + "="*60)
    print("BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA")
    print("="*60)
    
    print("\nStage 1: High-Frequency Labels")
    stage1_examples = extractor.build_few_shot_examples(
        train_posts, 
        extractor.STAGE_1,
        n_examples_per_label=3,
        annotator="a1"
    )

    print("\n" + "="*60)
    print("GENERATING PREDICTIONS ON TEST DATA")
    print("="*60)
    extractor.write_conll_predictions(
        test_posts, 
        PRED_PATH,
        stage1_examples,
    )
    
    print("\n" + "="*60)
    print("PREDICTION COMPLETE")
    print("="*60)
    print(f"\nTotal predictions made: {len(test_posts)} posts")
    print(f"Predictions saved to: {PRED_PATH}")
    print("="*60)

/home/ubuntu/environments/fadeit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


THREE-STAGE SPAN EXTRACTION

Initializing span extractor...
Loading model on device: cuda


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.26s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Parsing training CoNLL file...
Parsed 1152 posts from faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-b/train-dev.conll
Training posts: 1152

Parsing test CoNLL file...
Parsed 288 posts from faina-v1.0_fadeit-shared-task/test_data/subtask-b/test.conll
Test posts: 288

TRAINING SET LABEL DISTRIBUTION

LABEL DISTRIBUTION IN SPANS

High-Frequency Labels:
  Appeal-to-emotion: 792
  Vagueness: 426
  Doubt: 196
  Evading-the-burden-of-proof: 151
  False-analogy: 115
  Hasty-generalization: 191
  Strawman: 53
  Ad-hominem: 128
  Loaded-language: 863
  Red-herring: 103
  Causal-oversimplification: 68
  Cherry-picking: 41
  Slippery-slope: 72
  Name-calling-or-labelling: 446
  Flag-waving: 158
  False-dilemma: 35
  Thought-terminating-cliches: 114
  Appeal-to-authority: 96
  Slogan: 155
  Circular-reasoning: 10

Total spans: 4213

BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA

Stage 1: High-Frequency Labels

  Built examples for 20 labels in this tier:
 

### Three stage

In [ ]:
import sys
import json
import re
import os
from collections import defaultdict, Counter
from typing import List, Dict, Set
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


class ThreeStageSpanExtractor:
    
    STAGE_1 = [
        "Appeal-to-emotion",      
        "Vagueness",             
        "Doubt",                  
        "Evading-the-burden-of-proof",  
        "False-analogy",          
    ]
    
    STAGE_2 = [
        "Hasty-generalization",   
        "Strawman",               
        "Ad-hominem",             
        "Loaded-language",        
        "Red-herring",            
        "Causal-oversimplification",  
        "Cherry-picking",         
        "Slippery-slope",         
        "Name-calling-or-labelling",  
        "Flag-waving",            
        "False-dilemma",          
    ]
    
    STAGE_3 = [
        "Thought-terminating-cliches",  
        "Appeal-to-authority",    
        "Slogan",                 
        "Circular-reasoning",     
    ]
    
    ALL_LABELS = STAGE_1 + STAGE_2 + STAGE_3

    def __init__(self, model_name="meta-llama/Meta-Llama-3.1-8B-Instruct", use_cpu=False, use_4bit=False):
        self.device = "cuda" if torch.cuda.is_available() and not use_cpu else "cpu"
        print(f"Loading model on device: {self.device}")
    
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
    
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def parse_conll_file(self, filepath: str) -> List[Dict]:
        posts = []
        current_post = None
        
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.rstrip('\n')
                
                if line.startswith("# post_id = "):
                    if current_post:
                        posts.append(current_post)
                    current_post = {
                        'post_id': line.split(" = ")[1],
                        'date': '',
                        'topic': '',
                        'text': '',
                        'tokens': [],
                        'bio_tags_a1': [],
                        'bio_tags_a2': []
                    }
                elif line.startswith("# post_date = "):
                    current_post['date'] = line.split(" = ")[1]
                elif line.startswith("# post_topic_keywords = "):
                    current_post['topic'] = line.split(" = ")[1]
                elif line.startswith("# post_text = "):
                    current_post['text'] = line.split(" = ")[1]
                elif len(line.strip()) == 0:
                    if current_post and current_post['tokens']:
                        current_post['spans_a1'] = self._bio_to_spans(
                            current_post['tokens'], current_post['bio_tags_a1']
                        )
                        current_post['spans_a2'] = self._bio_to_spans(
                            current_post['tokens'], current_post['bio_tags_a2']
                        )
                        posts.append(current_post)
                        current_post = None
                elif current_post is not None:
                    parts = line.split('\t')
                    if len(parts) >= 4:
                        current_post['tokens'].append(parts[1])
                        current_post['bio_tags_a1'].append(parts[2])
                        current_post['bio_tags_a2'].append(parts[3])
        
        if current_post and current_post['tokens']:
            current_post['spans_a1'] = self._bio_to_spans(
                current_post['tokens'], current_post['bio_tags_a1']
            )
            current_post['spans_a2'] = self._bio_to_spans(
                current_post['tokens'], current_post['bio_tags_a2']
            )
            posts.append(current_post)
        
        print(f"Parsed {len(posts)} posts from {filepath}")
        return posts

    def _bio_to_spans(self, tokens: List[str], bio_tags: List[str]) -> List[Dict]:
        active_spans = {}
        completed_spans = []
        
        for idx, (token, bio_tag_str) in enumerate(zip(tokens, bio_tags)):
            bio_tags_list = bio_tag_str.split('|')
            current_labels = set()
            
            for bio_tag in bio_tags_list:
                bio_tag = bio_tag.strip()
                if bio_tag.startswith("B-"):
                    label = bio_tag[2:]
                    current_labels.add(label)
                    
                    if label in active_spans:
                        span = active_spans[label]
                        completed_spans.append({
                            'label': label,
                            'start_tok': span['start_tok'],
                            'end_tok': idx - 1,
                            'text': " ".join(span['tokens'])
                        })
                    
                    active_spans[label] = {'start_tok': idx, 'tokens': [token]}
                    
                elif bio_tag.startswith("I-"):
                    label = bio_tag[2:]
                    current_labels.add(label)
                    if label in active_spans:
                        active_spans[label]['tokens'].append(token)
                    else:
                        active_spans[label] = {'start_tok': idx, 'tokens': [token]}
            
            labels_to_close = set(active_spans.keys()) - current_labels
            for label in labels_to_close:
                span = active_spans[label]
                completed_spans.append({
                    'label': label,
                    'start_tok': span['start_tok'],
                    'end_tok': idx - 1,
                    'text': " ".join(span['tokens'])
                })
                del active_spans[label]
        
        for label, span in active_spans.items():
            completed_spans.append({
                'label': label,
                'start_tok': span['start_tok'],
                'end_tok': len(tokens) - 1,
                'text': " ".join(span['tokens'])
            })
        
        return completed_spans

    def _format_for_model(self, prompt: str) -> str:
        messages = [{"role": "user", "content": prompt}]
        if hasattr(self.tokenizer, "apply_chat_template"):
            try:
                return self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            except Exception:
                return prompt
        return prompt

    def _generate(self, prompt: str, max_tokens: int = 1024, temperature: float = 0.0) -> str:
        text = self._format_for_model(prompt)
        inputs = self.tokenizer(text, return_tensors="pt", padding=True).to(self.device)
        
        with torch.inference_mode():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=0.95,
                do_sample=True if temperature > 0 else False,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id or self.tokenizer.eos_token_id,
            )
        
        generated = outputs[0][inputs["input_ids"].shape[1]:]
        response = self.tokenizer.decode(generated, skip_special_tokens=True)
        
        del inputs, outputs, generated
        torch.cuda.empty_cache()
        
        return response

    def _extract_spans_from_response(self, response: str, valid_labels: List[str]) -> List[Dict]:
        spans = []
        
        # Clean response
        response = re.sub(r'```json\s*', '', response)
        response = re.sub(r'```\s*', '', response)
        response = response.strip()
        
        # Try JSON array
        try:
            start = response.find("[")
            end = response.rfind("]") + 1
            if start != -1 and end > start:
                json_str = response[start:end]
                data = json.loads(json_str)
                if isinstance(data, list):
                    spans = data
        except:
            pass
        
        # Try JSON object
        if not spans:
            try:
                start = response.find("{")
                end = response.rfind("}") + 1
                if start != -1 and end > start:
                    json_str = response[start:end]
                    data = json.loads(json_str)
                    if "spans" in data:
                        spans = data["spans"]
            except:
                pass
        
        # Line-by-line extraction
        if not spans:
            for line in response.split('\n'):
                text_match = re.search(r'"text"\s*:\s*"([^"]+)"', line)
                label_match = re.search(r'"label"\s*:\s*"([^"]+)"', line)
                if text_match and label_match:
                    spans.append({
                        "text": text_match.group(1),
                        "label": label_match.group(1)
                    })
        
        # Validate and filter spans
        valid_spans = []
        for span in spans:
            if not isinstance(span, dict):
                continue
            
            text = span.get("text", "").strip()
            label = span.get("label", "").strip().replace(" ", "-").replace("_", "-")
            
            if not text or not label:
                continue
            
            # Only accept labels from current stage
            if label in valid_labels:
                valid_spans.append({"text": text, "label": label})
            else:
                for valid_label in valid_labels:
                    if valid_label.lower() == label.lower():
                        valid_spans.append({"text": text, "label": valid_label})
                        break
        
        return valid_spans

    def build_few_shot_examples(self, posts: List[Dict], 
                                labels_subset: List[str],
                                n_examples_per_label: int = 3,
                                annotator: str = "a1") -> Dict[str, List[str]]:
        examples = defaultdict(list)
        spans_key = f'spans_{annotator}'
        
        # Collect spans per label
        label_spans = defaultdict(list)
        for post in posts:
            for span in post.get(spans_key, []):
                label = span['label']
                text = span['text'].strip()
                if label in labels_subset and len(text.split()) >= 2:
                    label_spans[label].append(text)
        
        # Select diverse examples
        for label, span_list in label_spans.items():
            unique_spans = list(set(span_list))
            sorted_spans = sorted(unique_spans, key=len)
            
            selected = []
            if len(sorted_spans) <= n_examples_per_label:
                selected = sorted_spans
            else:
                step = len(sorted_spans) // n_examples_per_label
                for i in range(n_examples_per_label):
                    idx = min(i * step, len(sorted_spans) - 1)
                    selected.append(sorted_spans[idx])
            
            examples[label] = selected
        
        print(f"\n  Built examples for {len(examples)} labels in this tier:")
        for label in labels_subset:
            count = len(examples.get(label, []))
            print(f"    {label}: {count} examples")
        
        return dict(examples)

    def create_stage_prompt(self, text: str, stage_labels: List[str],
                               examples: Dict[str, List[str]],
                               stage_name: str,
                               max_examples_per_label: int = 3) -> str:

        label_descriptions = {
            "Ad-hominem": "Attacking the speaker instead of the argument",
            "Appeal-to-authority": "Claiming something is true just because an authority says so",
            "Appeal-to-emotion": "Manipulating feelings instead of providing logic",
            "Causal-oversimplification": "Reducing complex cause-effect relationships to a simple claim",
            "Cherry-picking": "Mentioning only evidence that supports a claim",
            "Circular-reasoning": "The conclusion is also used as a premise",
            "Doubt": "Casting suspicion without offering evidence",
            "Evading-the-burden-of-proof": "Making claims without backing them up",
            "False-analogy": "Comparing two unrelated things as if they are similar",
            "False-dilemma": "Claiming only two options exist when more do",
            "Flag-waving": "Appealing to patriotism or national identity",
            "Hasty-generalization": "Concluding based on very limited data",
            "Loaded-language": "Using emotionally charged language",
            "Name-calling-or-labelling": "Using insult or label instead of argument",
            "Red-herring": "Bringing up irrelevant information to distract",
            "Slippery-slope": "Claiming one event will definitely lead to a dramatic outcome",
            "Slogan": "Repeating a catchy phrase instead of reasoning",
            "Strawman": "Distorting someone's argument to attack it more easily",
            "Thought-terminating-cliches": "Phrases that shut down critical thinking",
            "Vagueness": "Statements that are unclear and undefined"

        }

        prompt = f"""You are an expert in identifying logical fallacies in text.

    STAGE: {stage_name}

    Target Fallacies:
    """
        for label in stage_labels:
            prompt += f"- {label}: {label_descriptions.get(label, 'Propaganda technique')}\n"

        if examples:
            prompt += "\nExamples:\n"
            for label in stage_labels:
                if label in examples and examples[label]:
                    prompt += f"\n{label}:\n"
                    for ex in examples[label][:max_examples_per_label]:
                        prompt += f'  - "{ex}"\n'

        prompt += f"""
    TEXT TO ANALYZE:
    "{text}"

    ANALYSIS PROCESS:
    1. First, read the entire text carefully
    2. For each sentence/phrase, ask: "Does this match any of the {len(stage_labels)} target fallacies?"
    3. If yes, identify which specific fallacy and extract the exact span
    4. Ignore any fallacies NOT in the target list above
    5. Compile all matches

    Think through your analysis step by step, then provide the final JSON output:

    {{"reasoning": "brief explanation of what you found", "spans": [{{"text": "exact span", "label": "Label-name"}}]}}

    Your response:"""

        return prompt


    def predict_stage(self, text: str, stage_labels: List[str],
                     examples: Dict[str, List[str]], stage_name: str) -> List[Dict]:
        prompt = self.create_stage_prompt(text, stage_labels, examples, stage_name)
        response = self._generate(prompt, max_tokens=1024, temperature=0.0)
        spans = self._extract_spans_from_response(response, stage_labels)
        return spans

    def predict_three_stages(self, text: str, post_id: str,
                            stage1_examples: Dict[str, List[str]],
                            stage2_examples: Dict[str, List[str]],
                            stage3_examples: Dict[str, List[str]],
                            verbose: bool = True) -> List[Dict]:
        
        all_spans = []
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"Post ID: {post_id}")
            print(f"Text: {text[:200]}...")
        
        # Stage 1: High-frequency labels
        if verbose:
            print(f"\n  STAGE 1: High-frequency fallacies")
        stage1_spans = self.predict_stage(
            text, self.STAGE_1, stage1_examples, "High-Frequency"
        )
        if verbose:
            print(f"    Found {len(stage1_spans)} spans")
            for span in stage1_spans[:3]:
                print(f"      [{span['label']}] \"{span['text'][:50]}...\"")
        all_spans.extend(stage1_spans)
        
        # Stage 2: Mid-frequency labels
        if verbose:
            print(f"\n  STAGE 2: Mid-frequency fallacies")
        stage2_spans = self.predict_stage(
            text, self.STAGE_2, stage2_examples, "Mid-Frequency"
        )
        if verbose:
            print(f"    Found {len(stage2_spans)} spans")
            for span in stage2_spans[:3]:
                print(f"      [{span['label']}] \"{span['text'][:50]}...\"")
        all_spans.extend(stage2_spans)
        
        # Stage 3: Low-frequency labels
        if verbose:
            print(f"\n  STAGE 3: Low-frequency fallacies")
        stage3_spans = self.predict_stage(
            text, self.STAGE_3, stage3_examples, "Low-Frequency"
        )
        if verbose:
            print(f"    Found {len(stage3_spans)} spans")
            for span in stage3_spans[:3]:
                print(f"      [{span['label']}] \"{span['text'][:50]}...\"")
        all_spans.extend(stage3_spans)
        
        if verbose:
            print(f"\n  TOTAL: {len(all_spans)} spans across all stages")
            print(f"{'='*60}")
        
        return all_spans

    def spans_to_bio(self, text: str, tokens: List[str], spans: List[Dict]) -> List[str]:
        token_labels = [set() for _ in tokens]
        reconstructed = " ".join(tokens)
        reconstructed_lower = reconstructed.lower()
        
        for span in spans:
            span_text = span['text'].strip()
            label = span['label']
            span_text_lower = span_text.lower()
            
            span_start = reconstructed_lower.find(span_text_lower)
            
            if span_start != -1:
                prefix = reconstructed[:span_start]
                tokens_before = len(prefix.split()) if prefix.strip() else 0
                span_token_count = len(span_text.split())
                
                start_idx = tokens_before
                end_idx = min(start_idx + span_token_count, len(tokens))
                
                if start_idx < len(tokens):
                    token_labels[start_idx].add(f"B-{label}")
                    for idx in range(start_idx + 1, end_idx):
                        token_labels[idx].add(f"I-{label}")
                    continue
            
            # Fuzzy matching fallback
            span_tokens = span_text_lower.split()
            if not span_tokens:
                continue
            
            tokens_lower = [t.lower() for t in tokens]
            best_match_start = -1
            best_match_score = 0.0
            
            for i in range(len(tokens_lower) - len(span_tokens) + 1):
                window = tokens_lower[i:i + len(span_tokens)]
                matches = sum(1 for w, s in zip(window, span_tokens) if w == s or w in s or s in w)
                score = matches / len(span_tokens)
                
                if score > best_match_score:
                    best_match_score = score
                    best_match_start = i
            
            if best_match_start >= 0 and best_match_score >= 0.5:
                end_idx = best_match_start + len(span_tokens)
                token_labels[best_match_start].add(f"B-{label}")
                for idx in range(best_match_start + 1, end_idx):
                    token_labels[idx].add(f"I-{label}")
        
        # Convert to BIO format
        bio_tags = []
        for label_set in token_labels:
            if not label_set:
                bio_tags.append("O")
            else:
                bio_tags.append("|".join(sorted(label_set)))
        
        # Fix BIO sequence
        bio_tags = self._fix_bio_sequence(bio_tags)
        
        return bio_tags

    def _fix_bio_sequence(self, bio_tags: List[str]) -> List[str]:
        fixed_tags = []
        
        for idx, tag in enumerate(bio_tags):
            if tag == "O":
                fixed_tags.append(tag)
                continue
            
            labels_with_b = set()
            labels_with_i = set()
            
            for single_tag in tag.split("|"):
                if single_tag.startswith("B-"):
                    labels_with_b.add(single_tag[2:])
                elif single_tag.startswith("I-"):
                    labels_with_i.add(single_tag[2:])
            
            final_labels = []
            
            # Add B- tags
            for label in sorted(labels_with_b):
                final_labels.append(f"B-{label}")
            
            # Process I- tags
            for label in sorted(labels_with_i):
                if label in labels_with_b:
                    continue
                
                if idx > 0:
                    prev_tag = fixed_tags[idx - 1]
                    if f"B-{label}" in prev_tag or f"I-{label}" in prev_tag:
                        final_labels.append(f"I-{label}")
                    else:
                        final_labels.append(f"B-{label}")
                else:
                    final_labels.append(f"B-{label}")
            
            if final_labels:
                fixed_tags.append("|".join(sorted(final_labels)))
            else:
                fixed_tags.append("O")
        
        return fixed_tags

    def _load_checkpoint(self, checkpoint_path: str) -> Set[str]:
        if os.path.exists(checkpoint_path):
            try:
                with open(checkpoint_path, 'r', encoding='utf-8') as f:
                    processed_ids = set(json.load(f))
                print(f"\nCheckpoint found: {len(processed_ids)} posts already processed")
                return processed_ids
            except:
                print("\nCheckpoint file corrupted, starting fresh")
                return set()
        return set()

    def _save_checkpoint(self, checkpoint_path: str, processed_ids: Set[str]):
        with open(checkpoint_path, 'w', encoding='utf-8') as f:
            json.dump(list(processed_ids), f)

    def write_conll_predictions(self, posts: List[Dict], output_path: str,
                               stage1_examples: Dict[str, List[str]],
                               stage2_examples: Dict[str, List[str]],
                               stage3_examples: Dict[str, List[str]]):
        
        checkpoint_path = output_path + ".checkpoint"
        processed_ids = self._load_checkpoint(checkpoint_path)
        
        # Determine write mode
        write_mode = 'a' if processed_ids else 'w'
        
        with open(output_path, write_mode, encoding='utf-8') as f:
            for idx, post in enumerate(posts):
                # Skip if already processed
                if post['post_id'] in processed_ids:
                    print(f"\nSkipping post {idx+1}/{len(posts)}: {post['post_id']} (already processed)")
                    continue
                
                print(f"\n{'#'*60}")
                print(f"Processing post {idx+1}/{len(posts)}: {post['post_id']}")
                print(f"{'#'*60}")
                
                # Write metadata
                f.write(f"# post_id = {post['post_id']}\n")
                f.write(f"# post_date = {post['date']}\n")
                f.write(f"# post_topic_keywords = {post['topic']}\n")
                f.write(f"# post_text = {post['text']}\n")
                
                # 3-stage prediction
                predicted_spans = self.predict_three_stages(
                    post['text'], post['post_id'],
                    stage1_examples, stage2_examples, stage3_examples,
                    verbose=True
                )
                
                # Convert to BIO
                bio_tags = self.spans_to_bio(post['text'], post['tokens'], predicted_spans)
                
                # Ensure correct length
                if len(bio_tags) != len(post['tokens']):
                    print(f"   WARNING: Length mismatch! Fixing...")
                    if len(bio_tags) < len(post['tokens']):
                        bio_tags.extend(['O'] * (len(post['tokens']) - len(bio_tags)))
                    else:
                        bio_tags = bio_tags[:len(post['tokens'])]
                
                # Write tokens
                for tok_idx, (token, bio_tag) in enumerate(zip(post['tokens'], bio_tags), start=1):
                    f.write(f"{tok_idx}\t{token}\t{bio_tag}\t{bio_tag}\n")
                f.write("\n")
                
                # Flush to disk immediately
                f.flush()
                
                # Update checkpoint
                processed_ids.add(post['post_id'])
                self._save_checkpoint(checkpoint_path, processed_ids)
        
        # Clean up checkpoint file on successful completion
        if len(processed_ids) == len(posts):
            if os.path.exists(checkpoint_path):
                os.remove(checkpoint_path)
                print(f"\nCheckpoint file removed (all posts processed)")
        
        print(f"\n{'='*60}")
        print(f"Predictions saved to: {output_path}")
        print(f"{'='*60}")

    def analyze_label_distribution(self, posts: List[Dict], annotator: str = "a1"):
        spans_key = f'spans_{annotator}'
        label_counts = Counter()
        
        for post in posts:
            for span in post.get(spans_key, []):
                label_counts[span['label']] += 1
        
        print("\n" + "="*60)
        print("LABEL DISTRIBUTION IN SPANS")
        print("="*60)
        
        print("\nHigh-Frequency Labels:")
        for label in self.STAGE_1:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print("\nMid-Frequency Labels:")
        for label in self.STAGE_2:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print("\nLow-Frequency Labels:")
        for label in self.STAGE_3:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print(f"\nTotal spans: {sum(label_counts.values())}")


if __name__ == "__main__":
    TRAIN_CONLL_PATH = "faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-b/train-dev.conll"
    TEST_CONLL_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-b/test.conll"
    PRED_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-b/B_2.conll"
    
    print("="*60)
    print("THREE-STAGE SPAN EXTRACTION")
    print("="*60)
    
    print("\nInitializing span extractor...")
    extractor = ThreeStageSpanExtractor(use_4bit=False)
    
    # Load training data
    print("\nParsing training CoNLL file...")
    train_posts = extractor.parse_conll_file(TRAIN_CONLL_PATH)
    print(f"Training posts: {len(train_posts)}")
    
    # Load test data (without ground truth labels)
    print("\nParsing test CoNLL file...")
    test_posts = extractor.parse_conll_file(TEST_CONLL_PATH)
    print(f"Test posts: {len(test_posts)}")
    
    # Analyze training set distribution
    print("\n" + "="*60)
    print("TRAINING SET LABEL DISTRIBUTION")
    print("="*60)
    extractor.analyze_label_distribution(train_posts)
    
    print("\n" + "="*60)
    print("BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA")
    print("="*60)
    
    print("\nStage 1: High-Frequency Labels")
    stage1_examples = extractor.build_few_shot_examples(
        train_posts, 
        extractor.STAGE_1,
        n_examples_per_label=3,
        annotator="a1"
    )
    
    print("\nStage 2: Mid-Frequency Labels")
    stage2_examples = extractor.build_few_shot_examples(
        train_posts,
        extractor.STAGE_2,
        n_examples_per_label=3,
        annotator="a1"
    )
    
    print("\nStage 3: Low-Frequency Labels")
    stage3_examples = extractor.build_few_shot_examples(
        train_posts,
        extractor.STAGE_3,
        n_examples_per_label=3,
        annotator="a1"
    )
    
    print("\n" + "="*60)
    print("GENERATING PREDICTIONS ON TEST DATA")
    print("="*60)
    extractor.write_conll_predictions(
        test_posts, 
        PRED_PATH,
        stage1_examples,
        stage2_examples,
        stage3_examples
    )
    
    print("\n" + "="*60)
    print("PREDICTION COMPLETE")
    print("="*60)
    print(f"\nTotal predictions made: {len(test_posts)} posts")
    print(f"Predictions saved to: {PRED_PATH}")
    print("="*60)

/home/ubuntu/environments/fadeit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


THREE-STAGE SPAN EXTRACTION

Initializing span extractor...
Loading model on device: cuda


Loading checkpoint shards: 100%|██████████| 8/8 [00:34<00:00,  4.37s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Parsing training CoNLL file...
Parsed 1152 posts from faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-b/train-dev.conll
Training posts: 1152

Parsing test CoNLL file...
Parsed 288 posts from faina-v1.0_fadeit-shared-task/test_data/subtask-b/test.conll
Test posts: 288

TRAINING SET LABEL DISTRIBUTION

LABEL DISTRIBUTION IN SPANS

High-Frequency Labels:
  Appeal-to-emotion: 792
  Vagueness: 426
  Doubt: 196
  Evading-the-burden-of-proof: 151
  False-analogy: 115

Mid-Frequency Labels:
  Hasty-generalization: 191
  Strawman: 53
  Ad-hominem: 128
  Loaded-language: 863
  Red-herring: 103
  Causal-oversimplification: 68
  Cherry-picking: 41
  Slippery-slope: 72
  Name-calling-or-labelling: 446
  Flag-waving: 158
  False-dilemma: 35

Low-Frequency Labels:
  Thought-terminating-cliches: 114
  Appeal-to-authority: 96
  Slogan: 155
  Circular-reasoning: 10

Total spans: 4213

BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA

Stage 1: High-Frequency Labels



### Four stage

In [1]:
import sys
import json
import re
import os
from collections import defaultdict, Counter
from typing import List, Dict, Set
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


class ThreeStageSpanExtractor:
    STAGE_1 = [
        "Appeal-to-emotion",      
        "Vagueness",             
        "Doubt",                            
    ]
    
    STAGE_2 = [
        "Evading-the-burden-of-proof",  
        "False-analogy",
        "Hasty-generalization", 
        "Loaded-language",      
    ]
    
    STAGE_3 = [
        "Strawman",               
        "Ad-hominem",              
        "Red-herring",            
        "Causal-oversimplification",  
        "Cherry-picking",         
        "Slippery-slope",             
             
    ]
    
    STAGE_4 = [
        "Name-calling-or-labelling",  
        "Flag-waving",            
        "False-dilemma",
        "Thought-terminating-cliches",  
        "Appeal-to-authority",    
        "Slogan",                 
        "Circular-reasoning",
    ]
    
    
    ALL_LABELS = STAGE_1 + STAGE_2 + STAGE_3 + STAGE_4

    def __init__(self, model_name="meta-llama/Meta-Llama-3.1-8B-Instruct", use_cpu=False, use_4bit=False):
        self.device = "cuda" if torch.cuda.is_available() and not use_cpu else "cpu"
        print(f"Loading model on device: {self.device}")
    
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
    
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def parse_conll_file(self, filepath: str) -> List[Dict]:
        posts = []
        current_post = None
        
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.rstrip('\n')
                
                if line.startswith("# post_id = "):
                    if current_post:
                        posts.append(current_post)
                    current_post = {
                        'post_id': line.split(" = ")[1],
                        'date': '',
                        'topic': '',
                        'text': '',
                        'tokens': [],
                        'bio_tags_a1': [],
                        'bio_tags_a2': []
                    }
                elif line.startswith("# post_date = "):
                    current_post['date'] = line.split(" = ")[1]
                elif line.startswith("# post_topic_keywords = "):
                    current_post['topic'] = line.split(" = ")[1]
                elif line.startswith("# post_text = "):
                    current_post['text'] = line.split(" = ")[1]
                elif len(line.strip()) == 0:
                    if current_post and current_post['tokens']:
                        current_post['spans_a1'] = self._bio_to_spans(
                            current_post['tokens'], current_post['bio_tags_a1']
                        )
                        current_post['spans_a2'] = self._bio_to_spans(
                            current_post['tokens'], current_post['bio_tags_a2']
                        )
                        posts.append(current_post)
                        current_post = None
                elif current_post is not None:
                    parts = line.split('\t')
                    if len(parts) >= 4:
                        current_post['tokens'].append(parts[1])
                        current_post['bio_tags_a1'].append(parts[2])
                        current_post['bio_tags_a2'].append(parts[3])
        
        if current_post and current_post['tokens']:
            current_post['spans_a1'] = self._bio_to_spans(
                current_post['tokens'], current_post['bio_tags_a1']
            )
            current_post['spans_a2'] = self._bio_to_spans(
                current_post['tokens'], current_post['bio_tags_a2']
            )
            posts.append(current_post)
        
        print(f"Parsed {len(posts)} posts from {filepath}")
        return posts

    def _bio_to_spans(self, tokens: List[str], bio_tags: List[str]) -> List[Dict]:
        active_spans = {}
        completed_spans = []
        
        for idx, (token, bio_tag_str) in enumerate(zip(tokens, bio_tags)):
            bio_tags_list = bio_tag_str.split('|')
            current_labels = set()
            
            for bio_tag in bio_tags_list:
                bio_tag = bio_tag.strip()
                if bio_tag.startswith("B-"):
                    label = bio_tag[2:]
                    current_labels.add(label)
                    
                    if label in active_spans:
                        span = active_spans[label]
                        completed_spans.append({
                            'label': label,
                            'start_tok': span['start_tok'],
                            'end_tok': idx - 1,
                            'text': " ".join(span['tokens'])
                        })
                    
                    active_spans[label] = {'start_tok': idx, 'tokens': [token]}
                    
                elif bio_tag.startswith("I-"):
                    label = bio_tag[2:]
                    current_labels.add(label)
                    if label in active_spans:
                        active_spans[label]['tokens'].append(token)
                    else:
                        active_spans[label] = {'start_tok': idx, 'tokens': [token]}
            
            labels_to_close = set(active_spans.keys()) - current_labels
            for label in labels_to_close:
                span = active_spans[label]
                completed_spans.append({
                    'label': label,
                    'start_tok': span['start_tok'],
                    'end_tok': idx - 1,
                    'text': " ".join(span['tokens'])
                })
                del active_spans[label]
        
        for label, span in active_spans.items():
            completed_spans.append({
                'label': label,
                'start_tok': span['start_tok'],
                'end_tok': len(tokens) - 1,
                'text': " ".join(span['tokens'])
            })
        
        return completed_spans

    def _format_for_model(self, prompt: str) -> str:
        messages = [{"role": "user", "content": prompt}]
        if hasattr(self.tokenizer, "apply_chat_template"):
            try:
                return self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            except Exception:
                return prompt
        return prompt

    def _generate(self, prompt: str, max_tokens: int = 1024, temperature: float = 0.0) -> str:
        text = self._format_for_model(prompt)
        inputs = self.tokenizer(text, return_tensors="pt", padding=True).to(self.device)
        
        with torch.inference_mode():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=0.95,
                do_sample=True if temperature > 0 else False,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id or self.tokenizer.eos_token_id,
            )
        
        generated = outputs[0][inputs["input_ids"].shape[1]:]
        response = self.tokenizer.decode(generated, skip_special_tokens=True)
        
        del inputs, outputs, generated
        torch.cuda.empty_cache()
        
        return response

    def _extract_spans_from_response(self, response: str, valid_labels: List[str]) -> List[Dict]:
        spans = []
        
        # Clean response
        response = re.sub(r'```json\s*', '', response)
        response = re.sub(r'```\s*', '', response)
        response = response.strip()
        
        # Try JSON array
        try:
            start = response.find("[")
            end = response.rfind("]") + 1
            if start != -1 and end > start:
                json_str = response[start:end]
                data = json.loads(json_str)
                if isinstance(data, list):
                    spans = data
        except:
            pass
        
        # Try JSON object
        if not spans:
            try:
                start = response.find("{")
                end = response.rfind("}") + 1
                if start != -1 and end > start:
                    json_str = response[start:end]
                    data = json.loads(json_str)
                    if "spans" in data:
                        spans = data["spans"]
            except:
                pass
        
        # Line-by-line extraction
        if not spans:
            for line in response.split('\n'):
                text_match = re.search(r'"text"\s*:\s*"([^"]+)"', line)
                label_match = re.search(r'"label"\s*:\s*"([^"]+)"', line)
                if text_match and label_match:
                    spans.append({
                        "text": text_match.group(1),
                        "label": label_match.group(1)
                    })
        
        # Validate and filter spans
        valid_spans = []
        for span in spans:
            if not isinstance(span, dict):
                continue
            
            text = span.get("text", "").strip()
            label = span.get("label", "").strip().replace(" ", "-").replace("_", "-")
            
            if not text or not label:
                continue
            
            # Only accept labels from current stage
            if label in valid_labels:
                valid_spans.append({"text": text, "label": label})
            else:
                for valid_label in valid_labels:
                    if valid_label.lower() == label.lower():
                        valid_spans.append({"text": text, "label": valid_label})
                        break
        
        return valid_spans

    def build_few_shot_examples(self, posts: List[Dict], 
                                labels_subset: List[str],
                                n_examples_per_label: int = 3,
                                annotator: str = "a1") -> Dict[str, List[str]]:
        examples = defaultdict(list)
        spans_key = f'spans_{annotator}'
        
        # Collect spans per label
        label_spans = defaultdict(list)
        for post in posts:
            for span in post.get(spans_key, []):
                label = span['label']
                text = span['text'].strip()
                if label in labels_subset and len(text.split()) >= 2:
                    label_spans[label].append(text)
        
        # Select diverse examples
        for label, span_list in label_spans.items():
            unique_spans = list(set(span_list))
            sorted_spans = sorted(unique_spans, key=len)
            
            selected = []
            if len(sorted_spans) <= n_examples_per_label:
                selected = sorted_spans
            else:
                step = len(sorted_spans) // n_examples_per_label
                for i in range(n_examples_per_label):
                    idx = min(i * step, len(sorted_spans) - 1)
                    selected.append(sorted_spans[idx])
            
            examples[label] = selected
        
        print(f"\n  Built examples for {len(examples)} labels in this tier:")
        for label in labels_subset:
            count = len(examples.get(label, []))
            print(f"    {label}: {count} examples")
        
        return dict(examples)

    def create_stage_prompt(self, text: str, stage_labels: List[str],
                               examples: Dict[str, List[str]],
                               stage_name: str,
                               max_examples_per_label: int = 3) -> str:

        label_descriptions = {
            "Ad-hominem": "Attacking the speaker instead of the argument",
            "Appeal-to-authority": "Claiming something is true just because an authority says so",
            "Appeal-to-emotion": "Manipulating feelings instead of providing logic",
            "Causal-oversimplification": "Reducing complex cause-effect relationships to a simple claim",
            "Cherry-picking": "Mentioning only evidence that supports a claim",
            "Circular-reasoning": "The conclusion is also used as a premise",
            "Doubt": "Casting suspicion without offering evidence",
            "Evading-the-burden-of-proof": "Making claims without backing them up",
            "False-analogy": "Comparing two unrelated things as if they are similar",
            "False-dilemma": "Claiming only two options exist when more do",
            "Flag-waving": "Appealing to patriotism or national identity",
            "Hasty-generalization": "Concluding based on very limited data",
            "Loaded-language": "Using emotionally charged language",
            "Name-calling-or-labelling": "Using insult or label instead of argument",
            "Red-herring": "Bringing up irrelevant information to distract",
            "Slippery-slope": "Claiming one event will definitely lead to a dramatic outcome",
            "Slogan": "Repeating a catchy phrase instead of reasoning",
            "Strawman": "Distorting someone's argument to attack it more easily",
            "Thought-terminating-cliches": "Phrases that shut down critical thinking",
            "Vagueness": "Statements that are unclear and undefined"

        }

        prompt = f"""You are an expert in identifying logical fallacies in text.

    STAGE: {stage_name}

    Target Fallacies:
    """
        for label in stage_labels:
            prompt += f"- {label}: {label_descriptions.get(label, 'Propaganda technique')}\n"

        if examples:
            prompt += "\nExamples:\n"
            for label in stage_labels:
                if label in examples and examples[label]:
                    prompt += f"\n{label}:\n"
                    for ex in examples[label][:max_examples_per_label]:
                        prompt += f'  - "{ex}"\n'

        prompt += f"""
    TEXT TO ANALYZE:
    "{text}"

    ANALYSIS PROCESS:
    1. First, read the entire text carefully
    2. For each sentence/phrase, ask: "Does this match any of the {len(stage_labels)} target fallacies?"
    3. If yes, identify which specific fallacy and extract the exact span
    4. Ignore any fallacies NOT in the target list above
    5. Compile all matches

    Think through your analysis step by step, then provide the final JSON output:

    {{"reasoning": "brief explanation of what you found", "spans": [{{"text": "exact span", "label": "Label-name"}}]}}

    Your response:"""

        return prompt


    def predict_stage(self, text: str, stage_labels: List[str],
                     examples: Dict[str, List[str]], stage_name: str) -> List[Dict]:
        prompt = self.create_stage_prompt(text, stage_labels, examples, stage_name)
        response = self._generate(prompt, max_tokens=1024, temperature=0.0)
        spans = self._extract_spans_from_response(response, stage_labels)
        return spans

    def predict_three_stages(self, text: str, post_id: str,
                            stage1_examples: Dict[str, List[str]],
                            stage2_examples: Dict[str, List[str]],
                            stage3_examples: Dict[str, List[str]],
                            stage4_examples: Dict[str, List[str]],
                            verbose: bool = True) -> List[Dict]:
        
        all_spans = []
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"Post ID: {post_id}")
            print(f"Text: {text[:200]}...")
        
        # Stage 1: High-frequency labels
        if verbose:
            print(f"\n  STAGE 1: High-frequency fallacies")
        stage1_spans = self.predict_stage(
            text, self.STAGE_1, stage1_examples, "High-Frequency"
        )
        if verbose:
            print(f"    Found {len(stage1_spans)} spans")
            for span in stage1_spans[:3]:
                print(f"      [{span['label']}] \"{span['text'][:50]}...\"")
        all_spans.extend(stage1_spans)
        
        # Stage 2: Mid-frequency labels
        if verbose:
            print(f"\n  STAGE 2: Mid-frequency fallacies")
        stage2_spans = self.predict_stage(
            text, self.STAGE_2, stage2_examples, "Mid-Frequency"
        )
        if verbose:
            print(f"    Found {len(stage2_spans)} spans")
            for span in stage2_spans[:3]:
                print(f"      [{span['label']}] \"{span['text'][:50]}...\"")
        all_spans.extend(stage2_spans)
        
        # Stage 3: Low-frequency labels
        if verbose:
            print(f"\n  STAGE 3: Low-frequency fallacies")
        stage3_spans = self.predict_stage(
            text, self.STAGE_3, stage3_examples, "Low-Frequency"
        )
        if verbose:
            print(f"    Found {len(stage3_spans)} spans")
            for span in stage3_spans[:3]:
                print(f"      [{span['label']}] \"{span['text'][:50]}...\"")
        all_spans.extend(stage3_spans)

        # Stage 4: Lowest-frequency labels
        if verbose:
            print(f"\n  STAGE 4: Lowest-frequency fallacies")
        stage4_spans = self.predict_stage(
            text, self.STAGE_4, stage4_examples, "Lowest-Frequency"
        )
        if verbose:
            print(f"    Found {len(stage4_spans)} spans")
            for span in stage4_spans[:3]:
                print(f"      [{span['label']}] \"{span['text'][:50]}...\"")
        all_spans.extend(stage4_spans)
        
        if verbose:
            print(f"\n  TOTAL: {len(all_spans)} spans across all stages")
            print(f"{'='*60}")
        
        return all_spans

    def spans_to_bio(self, text: str, tokens: List[str], spans: List[Dict]) -> List[str]:
        token_labels = [set() for _ in tokens]
        reconstructed = " ".join(tokens)
        reconstructed_lower = reconstructed.lower()
        
        for span in spans:
            span_text = span['text'].strip()
            label = span['label']
            span_text_lower = span_text.lower()
            
            span_start = reconstructed_lower.find(span_text_lower)
            
            if span_start != -1:
                prefix = reconstructed[:span_start]
                tokens_before = len(prefix.split()) if prefix.strip() else 0
                span_token_count = len(span_text.split())
                
                start_idx = tokens_before
                end_idx = min(start_idx + span_token_count, len(tokens))
                
                if start_idx < len(tokens):
                    token_labels[start_idx].add(f"B-{label}")
                    for idx in range(start_idx + 1, end_idx):
                        token_labels[idx].add(f"I-{label}")
                    continue
            
            # Fuzzy matching fallback
            span_tokens = span_text_lower.split()
            if not span_tokens:
                continue
            
            tokens_lower = [t.lower() for t in tokens]
            best_match_start = -1
            best_match_score = 0.0
            
            for i in range(len(tokens_lower) - len(span_tokens) + 1):
                window = tokens_lower[i:i + len(span_tokens)]
                matches = sum(1 for w, s in zip(window, span_tokens) if w == s or w in s or s in w)
                score = matches / len(span_tokens)
                
                if score > best_match_score:
                    best_match_score = score
                    best_match_start = i
            
            if best_match_start >= 0 and best_match_score >= 0.5:
                end_idx = best_match_start + len(span_tokens)
                token_labels[best_match_start].add(f"B-{label}")
                for idx in range(best_match_start + 1, end_idx):
                    token_labels[idx].add(f"I-{label}")
        
        # Convert to BIO format
        bio_tags = []
        for label_set in token_labels:
            if not label_set:
                bio_tags.append("O")
            else:
                bio_tags.append("|".join(sorted(label_set)))
        
        # Fix BIO sequence
        bio_tags = self._fix_bio_sequence(bio_tags)
        
        return bio_tags

    def _fix_bio_sequence(self, bio_tags: List[str]) -> List[str]:
        fixed_tags = []
        
        for idx, tag in enumerate(bio_tags):
            if tag == "O":
                fixed_tags.append(tag)
                continue
            
            labels_with_b = set()
            labels_with_i = set()
            
            for single_tag in tag.split("|"):
                if single_tag.startswith("B-"):
                    labels_with_b.add(single_tag[2:])
                elif single_tag.startswith("I-"):
                    labels_with_i.add(single_tag[2:])
            
            final_labels = []
            
            # Add B- tags
            for label in sorted(labels_with_b):
                final_labels.append(f"B-{label}")
            
            # Process I- tags
            for label in sorted(labels_with_i):
                if label in labels_with_b:
                    continue
                
                if idx > 0:
                    prev_tag = fixed_tags[idx - 1]
                    if f"B-{label}" in prev_tag or f"I-{label}" in prev_tag:
                        final_labels.append(f"I-{label}")
                    else:
                        final_labels.append(f"B-{label}")
                else:
                    final_labels.append(f"B-{label}")
            
            if final_labels:
                fixed_tags.append("|".join(sorted(final_labels)))
            else:
                fixed_tags.append("O")
        
        return fixed_tags

    def _load_checkpoint(self, checkpoint_path: str) -> Set[str]:
        if os.path.exists(checkpoint_path):
            try:
                with open(checkpoint_path, 'r', encoding='utf-8') as f:
                    processed_ids = set(json.load(f))
                print(f"\nCheckpoint found: {len(processed_ids)} posts already processed")
                return processed_ids
            except:
                print("\nCheckpoint file corrupted, starting fresh")
                return set()
        return set()

    def _save_checkpoint(self, checkpoint_path: str, processed_ids: Set[str]):
        with open(checkpoint_path, 'w', encoding='utf-8') as f:
            json.dump(list(processed_ids), f)

    def write_conll_predictions(self, posts: List[Dict], output_path: str,
                               stage1_examples: Dict[str, List[str]],
                               stage2_examples: Dict[str, List[str]],
                               stage3_examples: Dict[str, List[str]],
                               stage4_examples: Dict[str, List[str]]):
        
        checkpoint_path = output_path + ".checkpoint"
        processed_ids = self._load_checkpoint(checkpoint_path)
        
        # Determine write mode
        write_mode = 'a' if processed_ids else 'w'
        
        with open(output_path, write_mode, encoding='utf-8') as f:
            for idx, post in enumerate(posts):
                # Skip if already processed
                if post['post_id'] in processed_ids:
                    print(f"\nSkipping post {idx+1}/{len(posts)}: {post['post_id']} (already processed)")
                    continue
                
                print(f"\n{'#'*60}")
                print(f"Processing post {idx+1}/{len(posts)}: {post['post_id']}")
                print(f"{'#'*60}")
                
                # Write metadata
                f.write(f"# post_id = {post['post_id']}\n")
                f.write(f"# post_date = {post['date']}\n")
                f.write(f"# post_topic_keywords = {post['topic']}\n")
                f.write(f"# post_text = {post['text']}\n")
                
                # 3-stage prediction
                predicted_spans = self.predict_three_stages(
                    post['text'], post['post_id'],
                    stage1_examples, stage2_examples, stage3_examples, stage4_examples,
                    verbose=True
                )
                
                # Convert to BIO
                bio_tags = self.spans_to_bio(post['text'], post['tokens'], predicted_spans)
                
                # Ensure correct length
                if len(bio_tags) != len(post['tokens']):
                    print(f"   WARNING: Length mismatch! Fixing...")
                    if len(bio_tags) < len(post['tokens']):
                        bio_tags.extend(['O'] * (len(post['tokens']) - len(bio_tags)))
                    else:
                        bio_tags = bio_tags[:len(post['tokens'])]
                
                # Write tokens
                for tok_idx, (token, bio_tag) in enumerate(zip(post['tokens'], bio_tags), start=1):
                    f.write(f"{tok_idx}\t{token}\t{bio_tag}\t{bio_tag}\n")
                f.write("\n")
                
                # Flush to disk immediately
                f.flush()
                
                # Update checkpoint
                processed_ids.add(post['post_id'])
                self._save_checkpoint(checkpoint_path, processed_ids)
        
        # Clean up checkpoint file on successful completion
        if len(processed_ids) == len(posts):
            if os.path.exists(checkpoint_path):
                os.remove(checkpoint_path)
                print(f"\nCheckpoint file removed (all posts processed)")
        
        print(f"\n{'='*60}")
        print(f"Predictions saved to: {output_path}")
        print(f"{'='*60}")

    def analyze_label_distribution(self, posts: List[Dict], annotator: str = "a1"):
        spans_key = f'spans_{annotator}'
        label_counts = Counter()
        
        for post in posts:
            for span in post.get(spans_key, []):
                label_counts[span['label']] += 1
        
        print("\n" + "="*60)
        print("LABEL DISTRIBUTION IN SPANS")
        print("="*60)
        
        print("\nHigh-Frequency Labels:")
        for label in self.STAGE_1:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print("\nMid-Frequency Labels:")
        for label in self.STAGE_2:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print("\nLow-Frequency Labels:")
        for label in self.STAGE_3:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")

        print("\nLowest-Frequency Labels:")
        for label in self.STAGE_4:
            count = label_counts.get(label, 0)
            print(f"  {label}: {count}")
        
        print(f"\nTotal spans: {sum(label_counts.values())}")


if __name__ == "__main__":
    TRAIN_CONLL_PATH = "faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-b/train-dev.conll"
    TEST_CONLL_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-b/test.conll"
    PRED_PATH = "faina-v1.0_fadeit-shared-task/test_data/subtask-b/B_3.conll"
    
    print("="*60)
    print("THREE-STAGE SPAN EXTRACTION")
    print("="*60)
    
    print("\nInitializing span extractor...")
    extractor = ThreeStageSpanExtractor(use_4bit=False)
    
    # Load training data
    print("\nParsing training CoNLL file...")
    train_posts = extractor.parse_conll_file(TRAIN_CONLL_PATH)
    print(f"Training posts: {len(train_posts)}")
    
    # Load test data (without ground truth labels)
    print("\nParsing test CoNLL file...")
    test_posts = extractor.parse_conll_file(TEST_CONLL_PATH)
    print(f"Test posts: {len(test_posts)}")
    
    # Analyze training set distribution
    print("\n" + "="*60)
    print("TRAINING SET LABEL DISTRIBUTION")
    print("="*60)
    extractor.analyze_label_distribution(train_posts)
    
    print("\n" + "="*60)
    print("BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA")
    print("="*60)
    
    print("\nStage 1: High-Frequency Labels")
    stage1_examples = extractor.build_few_shot_examples(
        train_posts, 
        extractor.STAGE_1,
        n_examples_per_label=3,
        annotator="a1"
    )
    
    print("\nStage 2: Mid-Frequency Labels")
    stage2_examples = extractor.build_few_shot_examples(
        train_posts,
        extractor.STAGE_2,
        n_examples_per_label=3,
        annotator="a1"
    )
    
    print("\nStage 3: Low-Frequency Labels")
    stage3_examples = extractor.build_few_shot_examples(
        train_posts,
        extractor.STAGE_3,
        n_examples_per_label=3,
        annotator="a1"
    )

    print("\nStage 4: Lowest-Frequency Labels")
    stage4_examples = extractor.build_few_shot_examples(
        train_posts,
        extractor.STAGE_4,
        n_examples_per_label=3,
        annotator="a1"
    )
    
    print("\n" + "="*60)
    print("GENERATING PREDICTIONS ON TEST DATA")
    print("="*60)
    extractor.write_conll_predictions(
        test_posts, 
        PRED_PATH,
        stage1_examples,
        stage2_examples,
        stage3_examples,
        stage4_examples
    )
    
    print("\n" + "="*60)
    print("PREDICTION COMPLETE")
    print("="*60)
    print(f"\nTotal predictions made: {len(test_posts)} posts")
    print(f"Predictions saved to: {PRED_PATH}")
    print("="*60)

/home/ubuntu/environments/fadeit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


THREE-STAGE SPAN EXTRACTION

Initializing span extractor...
Loading model on device: cuda


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.06s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Parsing training CoNLL file...
Parsed 1152 posts from faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-b/train-dev.conll
Training posts: 1152

Parsing test CoNLL file...
Parsed 288 posts from faina-v1.0_fadeit-shared-task/test_data/subtask-b/test.conll
Test posts: 288

TRAINING SET LABEL DISTRIBUTION

LABEL DISTRIBUTION IN SPANS

High-Frequency Labels:
  Appeal-to-emotion: 792
  Vagueness: 426
  Doubt: 196

Mid-Frequency Labels:
  Evading-the-burden-of-proof: 151
  False-analogy: 115
  Hasty-generalization: 191
  Loaded-language: 863

Low-Frequency Labels:
  Strawman: 53
  Ad-hominem: 128
  Red-herring: 103
  Causal-oversimplification: 68
  Cherry-picking: 41
  Slippery-slope: 72

Lowest-Frequency Labels:
  Name-calling-or-labelling: 446
  Flag-waving: 158
  False-dilemma: 35
  Thought-terminating-cliches: 114
  Appeal-to-authority: 96
  Slogan: 155
  Circular-reasoning: 10

Total spans: 4213

BUILDING STAGE-SPECIFIC FEW-SHOT EXAMPLES FROM TRAINING DATA

Stage 